In [1]:
import torch
import os
from torch import device
import re
import pdb
import pandas as pd

In [2]:
def get_args(path):
    model_name = path.split('search')[-1].split('_')[1]
    split_path = path.split('seed')[-1].split('_')
    seed = split_path[1]
    meta_batch_size = split_path[2]
    gs = split_path[3]
    args = {}
    try:
        finetune = ' '.join(split_path[4].split('-')[:-1])
       
        args = {'model_name':model_name, 'seed':seed, 'meta_batch':meta_batch_size, 'gs':gs, 'finetune':finetune}
    except:
        return None
  
    return args

In [3]:
def read_result_file(path):
    for i, l in enumerate(open(path)):
        if i == 1:
            l = l.split()
            result = float(l[-1])
            return result
        

In [4]:

def get_result_list(task, set_eval='test_eval_acc'):
    experiment_path = '/home/t-yaqingwang/azure_storage/projects/few_shot/'
    path = os.path.join(experiment_path, task)
    result_path = os.path.join(path, 'pt-results')
    result_list = []
    print(len(os.listdir(result_path)))
    for file_name in os.listdir(result_path):
        #args_dict = get_args(file_name) 

        eval_result = os.path.join(result_path, file_name)

        try:
            args_dict = get_log(eval_result, args_list, set_eval)
            #print(args_dict)
            result_list.append(args_dict)
        except:
            continue
    return result_list


In [5]:
def get_log(path, args_list=None, set_eval=None):
    
    log_path = os.path.join(path, 'log')
    line = open(log_path).readlines()[0]
    line =  re.sub(r'\<.*\>', 'None', line)
    args = eval(line)
   
   
    new_args = {}
    
   
    if args_list is not None:
        for arg_para in args_list:
            if arg_para not in args:
                print("error "+str(arg_para))
            new_args[arg_para] = args[arg_para]
    else:
        new_args = args
  
   
    for arg in args:
        if set_eval in arg:
            new_args[arg] = args[arg]
    return new_args
    

In [ ]:
CLUE_prompt_qqp
CLUE_prompt_rte_exp
CLUE_prompt_mpqa

# QQP

In [16]:

result_list = []
task_list = ['CLUE_prompt_qqp']

args_list = [ 'qqp_test_eval_acc', 'un_train_batch_size', 'qqp-clue_test_eval_acc',  'adapter_dim',
            'model_name_or_path', 'finetune_teacher_epoch', 'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list:
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()





90


In [17]:
result_frame

,qqp_test_eval_acc,un_train_batch_size,qqp-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
0,0.692629,4,0.642857,-1,roberta-large,20,20,3,50.0,4
1,0.785976,4,0.766667,-1,microsoft/deberta-large,20,30,1,50.0,4
2,0.724586,4,0.719048,-1,roberta-base,20,30,1,50.0,4
3,0.512689,4,0.452381,-1,microsoft/deberta-base,20,10,3,50.0,4
4,0.629557,4,0.633333,-1,bert-large-uncased,20,30,3,50.0,4
...,...,...,...,...,...,...,...,...,...,...
85,0.642370,4,0.676190,-1,microsoft/deberta-large,20,10,2,50.0,4
86,0.664408,4,0.676190,-1,microsoft/deberta-base,20,30,4,50.0,4
87,0.485481,4,0.485714,-1,bert-large-uncased,20,10,3,50.0,4
88,0.659115,4,0.704762,-1,microsoft/deberta-base,20,30,3,50.0,4


In [19]:
for model in ['bert-base-uncased', 'bert-large-uncased', 'roberta-base', 'roberta-large', 'microsoft/deberta-base', 'microsoft/deberta-large']:
    for k in [10, 20, 30]:
        for epoch in [50]:
            for BS in [4]:
                print("shot is {}, epoch is {}, batch size is {} and model name is {}".format(k, epoch, BS, model))
                display(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)])

              
                print(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['qqp_test_eval_acc'].mean() * 100, 2),  \
                      "("+str(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['qqp_test_eval_acc'].std()*100,2) )+")")
                
                print(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['qqp-clue_test_eval_acc'].mean()*100, 2),  \
                      "("+str(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['qqp-clue_test_eval_acc'].std()*100, 2))+ ")")

                print('')

shot is 10, epoch is 50, batch size is 4 and model name is bert-base-uncased


,qqp_test_eval_acc,un_train_batch_size,qqp-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
17,0.598565,4,0.676190,-1,bert-base-uncased,20,10,5,50.0,4
54,0.636656,4,0.614286,-1,bert-base-uncased,20,10,1,50.0,4
70,0.592877,4,0.566667,-1,bert-base-uncased,20,10,4,50.0,4
73,0.628840,4,0.652381,-1,bert-base-uncased,20,10,2,50.0,4
75,0.554316,4,0.552381,-1,bert-base-uncased,20,10,3,50.0,4


60.23 (3.27)
61.24 (5.33)

shot is 20, epoch is 50, batch size is 4 and model name is bert-base-uncased


,qqp_test_eval_acc,un_train_batch_size,qqp-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
31,0.553525,4,0.557143,-1,bert-base-uncased,20,20,5,50.0,4
36,0.628345,4,0.628571,-1,bert-base-uncased,20,20,4,50.0,4
49,0.649740,4,0.657143,-1,bert-base-uncased,20,20,1,50.0,4
59,0.618402,4,0.647619,-1,bert-base-uncased,20,20,3,50.0,4
69,0.652906,4,0.695238,-1,bert-base-uncased,20,20,2,50.0,4


62.06 (4.02)
63.71 (5.09)

shot is 30, epoch is 50, batch size is 4 and model name is bert-base-uncased


...

67.65 (4.27)
68.57 (3.73)

shot is 20, epoch is 50, batch size is 4 and model name is microsoft/deberta-large


,qqp_test_eval_acc,un_train_batch_size,qqp-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
13,0.724338,4,0.690476,-1,microsoft/deberta-large,20,20,3,50.0,4
18,0.684145,4,0.638095,-1,microsoft/deberta-large,20,20,4,50.0,4
42,0.657804,4,0.652381,-1,microsoft/deberta-large,20,20,1,50.0,4
44,0.607074,4,0.623810,-1,microsoft/deberta-large,20,20,5,50.0,4
55,0.754737,4,0.771429,-1,microsoft/deberta-large,20,20,2,50.0,4


68.56 (5.75)
67.52 (5.92)

shot is 30, epoch is 50, batch size is 4 and model name is microsoft/deberta-large


,qqp_test_eval_acc,un_train_batch_size,qqp-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
1,0.785976,4,0.766667,-1,microsoft/deberta-large,20,30,1,50.0,4
47,0.694633,4,0.671429,-1,microsoft/deberta-large,20,30,4,50.0,4
62,0.728123,4,0.738095,-1,microsoft/deberta-large,20,30,3,50.0,4
65,0.645981,4,0.652381,-1,microsoft/deberta-large,20,30,5,50.0,4
80,0.763171,4,0.733333,-1,microsoft/deberta-large,20,30,2,50.0,4


72.36 (5.55)
71.24 (4.83)



# RTE

In [21]:

result_list = []
task_list = ['CLUE_prompt_rte_exp']

args_list = [ 'rte_test_eval_acc', 'un_train_batch_size', 'rte-clue_test_eval_acc',  'adapter_dim',
            'model_name_or_path', 'finetune_teacher_epoch', 'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list:
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()

for model in ['bert-base-uncased', 'bert-large-uncased', 'roberta-base', 'roberta-large', 'microsoft/deberta-base', 'microsoft/deberta-large']:
    for k in [10, 20, 30]:
        for epoch in [50]:
            for BS in [4]:
                print("shot is {}, epoch is {}, batch size is {} and model name is {}".format(k, epoch, BS, model))
                display(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)])

              
                print(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['rte_test_eval_acc'].mean() * 100, 2),  \
                      "("+str(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['rte_test_eval_acc'].std()*100,2) )+")")
                
                print(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['rte-clue_test_eval_acc'].mean()*100, 2),  \
                      "("+str(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['rte-clue_test_eval_acc'].std()*100, 2))+ ")")

                print('')



90
shot is 10, epoch is 50, batch size is 4 and model name is bert-base-uncased


,rte_test_eval_acc,un_train_batch_size,rte-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
13,0.509025,4,0.528571,-1,bert-base-uncased,20,10,4,50.0,4
27,0.480144,4,0.504762,-1,bert-base-uncased,20,10,1,50.0,4
31,0.494585,4,0.476190,-1,bert-base-uncased,20,10,2,50.0,4
62,0.541516,4,0.538095,-1,bert-base-uncased,20,10,5,50.0,4
86,0.541516,4,0.542857,-1,bert-base-uncased,20,10,3,50.0,4


51.34 (2.77)
51.81 (2.76)

shot is 20, epoch is 50, batch size is 4 and model name is bert-base-uncased


,rte_test_eval_acc,un_train_batch_size,rte-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
33,0.487365,4,0.490476,-1,bert-base-uncased,20,20,2,50.0,4
35,0.490975,4,0.485714,-1,bert-base-uncased,20,20,5,50.0,4
42,0.516245,4,0.500000,-1,bert-base-uncased,20,20,1,50.0,4
56,0.498195,4,0.509524,-1,bert-base-uncased,20,20,4,50.0,4
72,0.530686,4,0.533333,-1,bert-base-uncased,20,20,3,50.0,4


50.47 (1.83)
50.38 (1.89)

shot is 30, epoch is 50, batch size is 4 and model name is bert-base-uncased


...

59.21 (6.08)
59.52 (6.13)

shot is 20, epoch is 50, batch size is 4 and model name is microsoft/deberta-large


,rte_test_eval_acc,un_train_batch_size,rte-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
4,0.610108,4,0.590476,-1,microsoft/deberta-large,20,20,5,50.0,4
11,0.592058,4,0.604762,-1,microsoft/deberta-large,20,20,1,50.0,4
37,0.718412,4,0.719048,-1,microsoft/deberta-large,20,20,4,50.0,4
40,0.660650,4,0.652381,-1,microsoft/deberta-large,20,20,2,50.0,4
76,0.711191,4,0.695238,-1,microsoft/deberta-large,20,20,3,50.0,4


65.85 (5.73)
65.24 (5.56)

shot is 30, epoch is 50, batch size is 4 and model name is microsoft/deberta-large


,rte_test_eval_acc,un_train_batch_size,rte-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
0,0.714801,4,0.709524,-1,microsoft/deberta-large,20,30,3,50.0,4
30,0.685921,4,0.671429,-1,microsoft/deberta-large,20,30,5,50.0,4
63,0.581227,4,0.580952,-1,microsoft/deberta-large,20,30,1,50.0,4
69,0.743682,4,0.747619,-1,microsoft/deberta-large,20,30,2,50.0,4
80,0.707581,4,0.685714,-1,microsoft/deberta-large,20,30,4,50.0,4


68.66 (6.24)
67.9 (6.2)



# MPQA

In [22]:



result_list = []
task_list = ['CLUE_prompt_mpqa']

args_list = [ 'mpqa_test_eval_acc', 'un_train_batch_size', 'mpqa-clue_test_eval_acc',  'adapter_dim',
            'model_name_or_path', 'finetune_teacher_epoch', 'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list:
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()

for model in ['bert-base-uncased', 'bert-large-uncased', 'roberta-base', 'roberta-large', 'microsoft/deberta-base', 'microsoft/deberta-large']:
    for k in [10, 20, 30]:
        for epoch in [50]:
            for BS in [4]:
                print("shot is {}, epoch is {}, batch size is {} and model name is {}".format(k, epoch, BS, model))
                display(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)])

              
                print(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['mpqa_test_eval_acc'].mean() * 100, 2),  \
                      "("+str(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['mpqa_test_eval_acc'].std()*100,2) )+")")
                
                print(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['mpqa-clue_test_eval_acc'].mean()*100, 2),  \
                      "("+str(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['mpqa-clue_test_eval_acc'].std()*100, 2))+ ")")

                print('')



90
shot is 10, epoch is 50, batch size is 4 and model name is bert-base-uncased


,mpqa_test_eval_acc,un_train_batch_size,mpqa-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
0,0.7575,4,0.757143,-1,bert-base-uncased,20,10,1,50.0,4
3,0.7730,4,0.728571,-1,bert-base-uncased,20,10,2,50.0,4
6,0.5120,4,0.480952,-1,bert-base-uncased,20,10,3,50.0,4
9,0.5945,4,0.552381,-1,bert-base-uncased,20,10,4,50.0,4
12,0.5740,4,0.528571,-1,bert-base-uncased,20,10,5,50.0,4


64.22 (11.65)
60.95 (12.48)

shot is 20, epoch is 50, batch size is 4 and model name is bert-base-uncased


,mpqa_test_eval_acc,un_train_batch_size,mpqa-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
1,0.8185,4,0.800000,-1,bert-base-uncased,20,20,1,50.0,4
4,0.7300,4,0.671429,-1,bert-base-uncased,20,20,2,50.0,4
7,0.5610,4,0.519048,-1,bert-base-uncased,20,20,3,50.0,4
10,0.7585,4,0.733333,-1,bert-base-uncased,20,20,4,50.0,4
13,0.7115,4,0.685714,-1,bert-base-uncased,20,20,5,50.0,4


71.59 (9.56)
68.19 (10.4)

shot is 30, epoch is 50, batch size is 4 and model name is bert-base-uncased


...

51.35 (2.03)
50.48 (2.84)

shot is 20, epoch is 50, batch size is 4 and model name is microsoft/deberta-large


,mpqa_test_eval_acc,un_train_batch_size,mpqa-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
46,0.5880,4,0.552381,-1,microsoft/deberta-large,20,20,1,50.0,4
49,0.5670,4,0.547619,-1,microsoft/deberta-large,20,20,2,50.0,4
52,0.5085,4,0.490476,-1,microsoft/deberta-large,20,20,3,50.0,4
55,0.6005,4,0.609524,-1,microsoft/deberta-large,20,20,4,50.0,4
58,0.6110,4,0.590476,-1,microsoft/deberta-large,20,20,5,50.0,4


57.5 (4.06)
55.81 (4.58)

shot is 30, epoch is 50, batch size is 4 and model name is microsoft/deberta-large


,mpqa_test_eval_acc,un_train_batch_size,mpqa-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
47,0.6295,4,0.590476,-1,microsoft/deberta-large,20,30,1,50.0,4
50,0.5405,4,0.509524,-1,microsoft/deberta-large,20,30,2,50.0,4
53,0.5295,4,0.504762,-1,microsoft/deberta-large,20,30,3,50.0,4
56,0.6240,4,0.628571,-1,microsoft/deberta-large,20,30,4,50.0,4
59,0.6350,4,0.633333,-1,microsoft/deberta-large,20,30,5,50.0,4


59.17 (5.21)
57.33 (6.27)



# SUBJ

In [23]:



result_list = []
task_list = ['CLUE_prompt_subj']

args_list = [ 'subj_test_eval_acc', 'un_train_batch_size', 'subj-clue_test_eval_acc',  'adapter_dim',
            'model_name_or_path', 'finetune_teacher_epoch', 'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list:
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()

for model in ['bert-base-uncased', 'bert-large-uncased', 'roberta-base', 'roberta-large', 'microsoft/deberta-base', 'microsoft/deberta-large']:
    for k in [10, 20, 30]:
        for epoch in [50]:
            for BS in [4]:
                print("shot is {}, epoch is {}, batch size is {} and model name is {}".format(k, epoch, BS, model))
                display(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)])

              
                print(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['subj_test_eval_acc'].mean() * 100, 2),  \
                      "("+str(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['subj_test_eval_acc'].std()*100,2) )+")")
                
                print(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['subj-clue_test_eval_acc'].mean()*100, 2),  \
                      "("+str(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)&(result_frame['num_train_epochs']==epoch)
                                    &(result_frame['per_device_train_batch_size']==BS)]['subj-clue_test_eval_acc'].std()*100, 2))+ ")")

                print('')



90
shot is 10, epoch is 50, batch size is 4 and model name is bert-base-uncased


,subj_test_eval_acc,un_train_batch_size,subj-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
0,0.8145,4,0.790476,-1,bert-base-uncased,20,10,1,50.0,4
3,0.8225,4,0.785714,-1,bert-base-uncased,20,10,2,50.0,4
6,0.8285,4,0.828571,-1,bert-base-uncased,20,10,3,50.0,4
9,0.8315,4,0.833333,-1,bert-base-uncased,20,10,4,50.0,4
12,0.5820,4,0.590476,-1,bert-base-uncased,20,10,5,50.0,4


77.58 (10.85)
76.57 (10.03)

shot is 20, epoch is 50, batch size is 4 and model name is bert-base-uncased


,subj_test_eval_acc,un_train_batch_size,subj-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
1,0.8415,4,0.828571,-1,bert-base-uncased,20,20,1,50.0,4
4,0.8745,4,0.866667,-1,bert-base-uncased,20,20,2,50.0,4
7,0.8305,4,0.823810,-1,bert-base-uncased,20,20,3,50.0,4
10,0.8440,4,0.857143,-1,bert-base-uncased,20,20,4,50.0,4
13,0.7820,4,0.757143,-1,bert-base-uncased,20,20,5,50.0,4


83.45 (3.36)
82.67 (4.29)

shot is 30, epoch is 50, batch size is 4 and model name is bert-base-uncased


...

77.99 (12.65)
78.1 (12.82)

shot is 20, epoch is 50, batch size is 4 and model name is microsoft/deberta-large


,subj_test_eval_acc,un_train_batch_size,subj-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
46,0.9005,4,0.909524,-1,microsoft/deberta-large,20,20,1,50.0,4
49,0.8920,4,0.909524,-1,microsoft/deberta-large,20,20,2,50.0,4
52,0.9305,4,0.942857,-1,microsoft/deberta-large,20,20,3,50.0,4
55,0.8900,4,0.880952,-1,microsoft/deberta-large,20,20,4,50.0,4
58,0.8920,4,0.876190,-1,microsoft/deberta-large,20,20,5,50.0,4


90.1 (1.7)
90.38 (2.68)

shot is 30, epoch is 50, batch size is 4 and model name is microsoft/deberta-large


,subj_test_eval_acc,un_train_batch_size,subj-clue_test_eval_acc,adapter_dim,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
47,0.8860,4,0.885714,-1,microsoft/deberta-large,20,30,1,50.0,4
50,0.9155,4,0.919048,-1,microsoft/deberta-large,20,30,2,50.0,4
53,0.9185,4,0.919048,-1,microsoft/deberta-large,20,30,3,50.0,4
56,0.9350,4,0.923810,-1,microsoft/deberta-large,20,30,4,50.0,4
59,0.9365,4,0.933333,-1,microsoft/deberta-large,20,30,5,50.0,4


91.83 (2.04)
91.62 (1.8)



# Embedding Part

In [27]:


result_list = []
task_list = ['CLUE_SST_component_embedding']

args_list = [ 'sst-2_test_eval_acc', 'un_train_batch_size', 'sst-2-clue_test_eval_acc',  'adapter_dim', 'update_component',
            'model_name_or_path', 'finetune_teacher_epoch', 'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list:
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()


15


In [28]:

for model in [ 'roberta-large']:
    for k in [10, 20, 30]:
     
        for update_component in ['embedding']:
            print("shot is {}, epoch is {}, update_component is {} and model name is {}".format(k, epoch, update_component, model))
            display(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)
                                &(result_frame['update_component']==update_component)])


            print(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)
                                &(result_frame['update_component']==update_component)]['sst-2_test_eval_acc'].mean() * 100, 2),  \
                  "("+str(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)
                                &(result_frame['update_component']==update_component)]['sst-2_test_eval_acc'].std()*100,2) )+")")

            print(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)
                                &(result_frame['update_component']==update_component)]['sst-2-clue_test_eval_acc'].mean()*100, 2),  \
                  "("+str(round(result_frame[(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)
                                &(result_frame['update_component']==update_component)]['sst-2-clue_test_eval_acc'].std()*100, 2))+ ")")

            print('')

shot is 10, epoch is 50, update_component is embedding and model name is roberta-large


,sst-2_test_eval_acc,un_train_batch_size,sst-2-clue_test_eval_acc,adapter_dim,update_component,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
0,0.872706,4,0.742857,-1,embedding,roberta-large,20,10,3,100.0,4
1,0.860092,4,0.804762,-1,embedding,roberta-large,20,10,2,100.0,4
7,0.790138,4,0.752381,-1,embedding,roberta-large,20,10,4,100.0,4
9,0.864679,4,0.785714,-1,embedding,roberta-large,20,10,1,100.0,4
14,0.858945,4,0.642857,-1,embedding,roberta-large,20,10,5,100.0,4


84.93 (3.35)
74.57 (6.27)

shot is 20, epoch is 50, update_component is embedding and model name is roberta-large


,sst-2_test_eval_acc,un_train_batch_size,sst-2-clue_test_eval_acc,adapter_dim,update_component,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
2,0.845183,4,0.785714,-1,embedding,roberta-large,20,20,4,100.0,4
3,0.864679,4,0.809524,-1,embedding,roberta-large,20,20,1,100.0,4
5,0.884174,4,0.757143,-1,embedding,roberta-large,20,20,5,100.0,4
6,0.857798,4,0.780952,-1,embedding,roberta-large,20,20,2,100.0,4
12,0.875000,4,0.804762,-1,embedding,roberta-large,20,20,3,100.0,4


86.54 (1.51)
78.76 (2.09)

shot is 30, epoch is 50, update_component is embedding and model name is roberta-large


,sst-2_test_eval_acc,un_train_batch_size,sst-2-clue_test_eval_acc,adapter_dim,update_component,model_name_or_path,finetune_teacher_epoch,num_k,seed,num_train_epochs,per_device_train_batch_size
4,0.855505,4,0.790476,-1,embedding,roberta-large,20,30,2,100.0,4
8,0.876147,4,0.819048,-1,embedding,roberta-large,20,30,3,100.0,4
10,0.862385,4,0.800000,-1,embedding,roberta-large,20,30,1,100.0,4
11,0.850917,4,0.800000,-1,embedding,roberta-large,20,30,4,100.0,4
13,0.880734,4,0.761905,-1,embedding,roberta-large,20,30,5,100.0,4


86.51 (1.29)
79.43 (2.09)



In [30]:


result_list = []
task_list = ['CLUE_MNLI_component_embedding']

args_list = [ 'mnli-mm_test_eval_mnli-mm/acc', 'mnli_test_eval_mnli/acc',  'update_component',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()



15


In [32]:
for model in [ 'roberta-large']:
    for k in [10, 20, 30]:
        for update_component in ['embedding']:
     
            print("shot is {}, epoch is {}, update_component is {} and model name is {}".format(k, epoch, update_component, model))
            display(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)])


            print(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli_test_eval_mnli/acc'].mean() * 100, 2),  \
                  "("+str(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli_test_eval_mnli/acc'].std()*100,2) )+")")

            print(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].mean()*100, 2),  \
                  "("+str(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].std()*100, 2))+ ")")

            print('')

shot is 10, epoch is 50, update_component is embedding and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
2,0.551668,0.543963,embedding,roberta-large,10,4,50.0,4
4,0.540785,0.538462,embedding,roberta-large,10,1,50.0,4
6,0.543633,0.536933,embedding,roberta-large,10,2,50.0,4
10,0.539361,0.535813,embedding,roberta-large,10,3,50.0,4
14,0.515867,0.513194,embedding,roberta-large,10,5,50.0,4


53.37 (1.19)
53.83 (1.34)

shot is 20, epoch is 50, update_component is embedding and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
0,0.521664,0.525420,embedding,roberta-large,20,1,50.0,4
1,0.526037,0.532450,embedding,roberta-large,20,2,50.0,4
3,0.559194,0.555069,embedding,roberta-large,20,3,50.0,4
7,0.533971,0.523179,embedding,roberta-large,20,4,50.0,4
12,0.523088,0.530820,embedding,roberta-large,20,5,50.0,4


53.34 (1.27)
53.28 (1.55)

shot is 30, epoch is 50, update_component is embedding and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
5,0.533360,0.539888,embedding,roberta-large,30,5,50.0,4
8,0.557669,0.554356,embedding,roberta-large,30,3,50.0,4
9,0.540073,0.534284,embedding,roberta-large,30,4,50.0,4
11,0.535598,0.539684,embedding,roberta-large,30,2,50.0,4
13,0.529496,0.533469,embedding,roberta-large,30,1,50.0,4


54.03 (0.84)
53.92 (1.1)



In [33]:


result_list = []
task_list = ['CLUE_finetune_adapter_prompt_mnli_adapterafter']

args_list = [ 'mnli-mm_test_eval_mnli-mm/acc', 'mnli_test_eval_mnli/acc',  'update_component',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()



15


In [34]:
for model in [ 'roberta-large']:
    for k in [10, 20, 30]:
        for update_component in ['adapter']:
     
            print("shot is {}, epoch is {}, update_component is {} and model name is {}".format(k, epoch, update_component, model))
            display(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)])


            print(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli_test_eval_mnli/acc'].mean() * 100, 2),  \
                  "("+str(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli_test_eval_mnli/acc'].std()*100,2) )+")")

            print(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].mean()*100, 2),  \
                  "("+str(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].std()*100, 2))+ ")")

            print('')

shot is 10, epoch is 50, update_component is adapter and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
0,0.350895,0.351197,adapter,roberta-large,10,1,400.0,4
3,0.341131,0.341824,adapter,roberta-large,10,2,400.0,4
6,0.345810,0.349567,adapter,roberta-large,10,3,400.0,4
9,0.326485,0.329292,adapter,roberta-large,10,4,400.0,4
12,0.339300,0.341009,adapter,roberta-large,10,5,400.0,4


34.26 (0.87)
34.07 (0.91)

shot is 20, epoch is 50, update_component is adapter and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
1,0.352014,0.348344,adapter,roberta-large,20,1,400.0,4
4,0.337266,0.332348,adapter,roberta-large,20,2,400.0,4
7,0.338283,0.339786,adapter,roberta-large,20,3,400.0,4
10,0.337469,0.332450,adapter,roberta-large,20,4,400.0,4
13,0.338487,0.346612,adapter,roberta-large,20,5,400.0,4


33.99 (0.76)
34.07 (0.63)

shot is 30, epoch is 50, update_component is adapter and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
2,0.353234,0.350484,adapter,roberta-large,30,1,400.0,4
5,0.336452,0.329496,adapter,roberta-large,30,2,400.0,4
8,0.346623,0.341518,adapter,roberta-large,30,3,400.0,4
11,0.346623,0.347937,adapter,roberta-large,30,4,400.0,4
14,0.348454,0.352827,adapter,roberta-large,30,5,400.0,4


34.45 (0.94)
34.63 (0.61)



In [ ]:
CLUE_finetune_component_interm

In [39]:


result_list = []
task_list = ['CLUE_finetune_adapter_prompt_mnli_new_init_2']

args_list = [ 'mnli-mm_test_eval_mnli-mm/acc', 'mnli_test_eval_mnli/acc',  'update_component',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()



15


In [40]:
for model in [ 'roberta-large']:
    for k in [10, 20, 30]:
        for update_component in ['adapter']:
     
            print("shot is {}, epoch is {}, update_component is {} and model name is {}".format(k, epoch, update_component, model))
            display(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)])


            print(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli_test_eval_mnli/acc'].mean() * 100, 2),  \
                  "("+str(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli_test_eval_mnli/acc'].std()*100,2) )+")")

            print(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].mean()*100, 2),  \
                  "("+str(round(result_frame[(result_frame['update_component']==update_component)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].std()*100, 2))+ ")")

            print('')

shot is 10, epoch is 50, update_component is adapter and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
0,0.582587,0.574427,adapter,roberta-large,10,1,400.0,4
3,0.560720,0.552827,adapter,roberta-large,10,2,400.0,4
6,0.581062,0.569740,adapter,roberta-large,10,3,400.0,4
9,0.599471,0.577789,adapter,roberta-large,10,4,400.0,4
12,0.484744,0.490372,adapter,roberta-large,10,5,400.0,4


55.3 (3.63)
56.17 (4.52)

shot is 20, epoch is 50, update_component is adapter and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
1,0.553702,0.553031,adapter,roberta-large,20,1,400.0,4
4,0.606692,0.592461,adapter,roberta-large,20,2,400.0,4
7,0.619304,0.614366,adapter,roberta-large,20,3,400.0,4
10,0.617779,0.586551,adapter,roberta-large,20,4,400.0,4
13,0.592860,0.585430,adapter,roberta-large,20,5,400.0,4


58.64 (2.2)
59.81 (2.7)

shot is 30, epoch is 50, update_component is adapter and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
2,0.588588,0.583800,adapter,roberta-large,30,1,400.0,4
5,0.662937,0.635965,adapter,roberta-large,30,2,400.0,4
8,0.621745,0.613143,adapter,roberta-large,30,3,400.0,4
11,0.621338,0.602038,adapter,roberta-large,30,4,400.0,4
14,0.616558,0.616403,adapter,roberta-large,30,5,400.0,4


61.03 (1.92)
62.22 (2.66)



In [43]:



result_list = []
task_list = ['CLUE_finetune_adapter_prompt_mnli_different_init', 'CLUE_finetune_adapter_prompt_mnli_different_init_small_rate']

args_list = [ 'mnli-mm_test_eval_mnli-mm/acc', 'adapter_init_std', 'mnli_test_eval_mnli/acc',  'update_component',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()



45
30


In [44]:
for model in [ 'roberta-large']:
    for k in [10, 20, 30]:
        for adapter_init_std in [0.00002, 0.002, 0.02, 0.1, 1]:
     
            print("shot is {}, epoch is {}, adapter_init_std is {} and model name is {}".format(k, epoch, adapter_init_std, model))
            display(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)])


            print(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli_test_eval_mnli/acc'].mean() * 100, 2),  \
                  "("+str(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli_test_eval_mnli/acc'].std()*100,2) )+")")

            print(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].mean()*100, 2),  \
                  "("+str(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].std()*100, 2))+ ")")

            print('')

shot is 10, epoch is 50, adapter_init_std is 2e-05 and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,adapter_init_std,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
46,0.582181,0.00002,0.574325,adapter,roberta-large,10,1,400.0,4
51,0.561025,0.00002,0.552318,adapter,roberta-large,10,2,400.0,4
56,0.580960,0.00002,0.569638,adapter,roberta-large,10,3,400.0,4
62,0.598657,0.00002,0.577891,adapter,roberta-large,10,4,400.0,4
68,0.485456,0.00002,0.490779,adapter,roberta-large,10,5,400.0,4


55.3 (3.61)
56.17 (4.46)

shot is 10, epoch is 50, adapter_init_std is 0.002 and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,adapter_init_std,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
45,0.588995,0.002,0.580846,adapter,roberta-large,10,1,400.0,4
61,0.606998,0.002,0.587672,adapter,roberta-large,10,4,400.0,4
67,0.478946,0.002,0.484259,adapter,roberta-large,10,5,400.0,4


55.09 (5.78)
55.83 (6.93)

shot is 10, epoch is 50, adapter_init_std is 0.02 and model name is roberta-large


...

43.19 (4.86)
44.17 (5.2)

shot is 30, epoch is 50, adapter_init_std is 0.1 and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,adapter_init_std,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
3,0.352014,0.1,0.354356,adapter,roberta-large,30,1,400.0,4
6,0.335435,0.1,0.334997,adapter,roberta-large,30,3,400.0,4
11,0.331469,0.1,0.326745,adapter,roberta-large,30,4,400.0,4
13,0.346623,0.1,0.352012,adapter,roberta-large,30,2,400.0,4
33,0.337673,0.1,0.344778,adapter,roberta-large,30,5,400.0,4


34.26 (1.16)
34.06 (0.84)

shot is 30, epoch is 50, adapter_init_std is 1 and model name is roberta-large


,mnli-mm_test_eval_mnli-mm/acc,adapter_init_std,mnli_test_eval_mnli/acc,update_component,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size
1,0.352217,1.0,0.354356,adapter,roberta-large,30,5,400.0,4
9,0.352217,1.0,0.354457,adapter,roberta-large,30,1,400.0,4
19,0.346115,1.0,0.349363,adapter,roberta-large,30,2,400.0,4
20,0.329536,1.0,0.327356,adapter,roberta-large,30,4,400.0,4
23,0.333605,1.0,0.332960,adapter,roberta-large,30,3,400.0,4


34.37 (1.27)
34.27 (1.06)



In [45]:





result_list = []
task_list = ['full_adapter_mnli']

args_list = [ 'mnli-mm_test_eval_mnli-mm/acc', 'adapter_init_std', 'mnli_test_eval_mnli/acc',  'update_component',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()



8


In [47]:
result_frame

""


In [46]:
for model in ['roberta-base', 'roberta-large']:
    for adapter_init_std in [0.0000001, 0.002, 0.2, 1]:

        print("shot is {}, epoch is {}, adapter_init_std is {} and model name is {}".format(k, epoch, adapter_init_std, model))
        display(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['model_name_or_path']==model)])


        print(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['model_name_or_path']==model)]['mnli_test_eval_mnli/acc'].mean() * 100, 2),  \
              "("+str(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['model_name_or_path']==model)]['mnli_test_eval_mnli/acc'].std()*100,2) )+")")

        print(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].mean()*100, 2),  \
              "("+str(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].std()*100, 2))+ ")")

        print('')

shot is 30, epoch is 50, adapter_init_std is 1e-07 and model name is roberta-base


KeyError: 'adapter_init_std'

In [73]:
result_list = []
task_list = ['CLUE_semi_prompt_metast_fourtasks']

args_list = [   'update_component', 'task_name',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()



217


In [74]:
result_frame

,update_component,task_name,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,qqp_test_eval_acc,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_acc,rte_test_eval_acc,rte-clue_test_eval_acc,mpqa_test_eval_acc,mpqa-clue_test_eval_acc,subj_test_eval_acc,subj-clue_test_eval_acc
0,none,qqp,roberta-large,10,1,10000.0,4,0.651496,0.494686,0.638095,NaN,NaN,NaN,NaN,NaN,NaN
1,none,qqp,roberta-large,20,1,10000.0,4,0.634974,0.492249,0.619048,NaN,NaN,NaN,NaN,NaN,NaN
2,none,qqp,roberta-large,10,2,10000.0,4,0.655627,0.564270,0.671429,NaN,NaN,NaN,NaN,NaN,NaN
3,none,qqp,roberta-large,20,2,10000.0,4,0.748875,0.707836,0.771429,NaN,NaN,NaN,NaN,NaN,NaN
4,none,qqp,roberta-large,30,2,10000.0,4,0.776453,0.751295,0.728571,NaN,NaN,NaN,NaN,NaN,NaN
5,none,qqp,roberta-large,20,3,10000.0,4,0.721247,0.614961,0.671429,NaN,NaN,NaN,NaN,NaN,NaN
6,none,qqp,roberta-large,30,3,10000.0,4,0.746970,0.664897,0.704762,NaN,NaN,NaN,NaN,NaN,NaN
7,none,qqp,roberta-large,20,4,10000.0,4,0.698046,0.546335,0.657143,NaN,NaN,NaN,NaN,NaN,NaN
8,none,qqp,roberta-large,30,4,10000.0,4,0.754811,0.673526,0.728571,NaN,NaN,NaN,NaN,NaN,NaN
9,none,qqp,roberta-large,20,5,10000.0,4,0.531388,0.528904,0.514286,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
for l in result_frame.columns[result_frame.columns.to_series().str.contains('qqp') & result_frame.columns.to_series().str.contains('test_eval_acc')].to_series():
    print(l)

qqp_test_eval_acc
qqp_test_eval_acc_and_f1
qqp-clue_test_eval_acc


In [76]:
for model in ['roberta-large']:
    for task_name in ['qqp', 'rte', 'subj', 'mpqa']:
        for k in [10, 20, 30]:

            print("shot is {}, epoch is {}, task_name is {} and model name is {}".format(k, epoch, task_name, model))
            display(result_frame[(result_frame['task_name']==task_name)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)])
        
            for  metric in result_frame.columns[result_frame.columns.to_series().str.contains(task_name) & result_frame.columns.to_series().str.contains('test_eval_acc')].to_series():
                
                print(metric, round(result_frame[(result_frame['task_name']==task_name)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)][metric].mean() * 100, 2),  \
                      "("+str(round(result_frame[(result_frame['task_name']==task_name)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)][metric].std()*100,2) )+")")
                

#             print(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].mean()*100, 2),  \
#                   "("+str(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].std()*100, 2))+ ")")

#             print('')

shot is 10, epoch is 50, task_name is qqp and model name is roberta-large


,update_component,task_name,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,qqp_test_eval_acc,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_acc,rte_test_eval_acc,rte-clue_test_eval_acc,mpqa_test_eval_acc,mpqa-clue_test_eval_acc,subj_test_eval_acc,subj-clue_test_eval_acc
0,none,qqp,roberta-large,10,1,10000.0,4,0.651496,0.494686,0.638095,NaN,NaN,NaN,NaN,NaN,NaN
2,none,qqp,roberta-large,10,2,10000.0,4,0.655627,0.564270,0.671429,NaN,NaN,NaN,NaN,NaN,NaN


qqp_test_eval_acc 65.36 (0.29)
qqp_test_eval_acc_and_f1 52.95 (4.92)
qqp-clue_test_eval_acc 65.48 (2.36)
shot is 20, epoch is 50, task_name is qqp and model name is roberta-large


,update_component,task_name,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,qqp_test_eval_acc,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_acc,rte_test_eval_acc,rte-clue_test_eval_acc,mpqa_test_eval_acc,mpqa-clue_test_eval_acc,subj_test_eval_acc,subj-clue_test_eval_acc
1,none,qqp,roberta-large,20,1,10000.0,4,0.634974,0.492249,0.619048,NaN,NaN,NaN,NaN,NaN,NaN
3,none,qqp,roberta-large,20,2,10000.0,4,0.748875,0.707836,0.771429,NaN,NaN,NaN,NaN,NaN,NaN
5,none,qqp,roberta-large,20,3,10000.0,4,0.721247,0.614961,0.671429,NaN,NaN,NaN,NaN,NaN,NaN
7,none,qqp,roberta-large,20,4,10000.0,4,0.698046,0.546335,0.657143,NaN,NaN,NaN,NaN,NaN,NaN
9,none,qqp,roberta-large,20,5,10000.0,4,0.531388,0.528904,0.514286,NaN,NaN,NaN,NaN,NaN,NaN


qqp_test_eval_acc 66.69 (8.66)
qqp_test_eval_acc_and_f1 57.81 (8.51)
qqp-clue_test_eval_acc 64.67 (9.3)
shot is 30, epoch is 50, task_name is qqp and model name is roberta-large


...

mpqa_test_eval_acc 67.05 (8.77)
mpqa-clue_test_eval_acc 64.86 (8.63)
shot is 20, epoch is 50, task_name is mpqa and model name is roberta-large


,update_component,task_name,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,qqp_test_eval_acc,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_acc,rte_test_eval_acc,rte-clue_test_eval_acc,mpqa_test_eval_acc,mpqa-clue_test_eval_acc,subj_test_eval_acc,subj-clue_test_eval_acc
27,none,mpqa,roberta-large,20,1,10000.0,4,NaN,NaN,NaN,NaN,NaN,0.8385,0.819048,NaN,NaN
30,none,mpqa,roberta-large,20,2,10000.0,4,NaN,NaN,NaN,NaN,NaN,0.8265,0.804762,NaN,NaN
33,none,mpqa,roberta-large,20,3,10000.0,4,NaN,NaN,NaN,NaN,NaN,0.6740,0.685714,NaN,NaN
36,none,mpqa,roberta-large,20,4,10000.0,4,NaN,NaN,NaN,NaN,NaN,0.8705,0.852381,NaN,NaN
39,none,mpqa,roberta-large,20,5,10000.0,4,NaN,NaN,NaN,NaN,NaN,0.8195,0.780952,NaN,NaN


mpqa_test_eval_acc 80.58 (7.62)
mpqa-clue_test_eval_acc 78.86 (6.3)
shot is 30, epoch is 50, task_name is mpqa and model name is roberta-large


,update_component,task_name,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,qqp_test_eval_acc,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_acc,rte_test_eval_acc,rte-clue_test_eval_acc,mpqa_test_eval_acc,mpqa-clue_test_eval_acc,subj_test_eval_acc,subj-clue_test_eval_acc
28,none,mpqa,roberta-large,30,1,10000.0,4,NaN,NaN,NaN,NaN,NaN,0.8740,0.885714,NaN,NaN
31,none,mpqa,roberta-large,30,2,10000.0,4,NaN,NaN,NaN,NaN,NaN,0.8125,0.809524,NaN,NaN
34,none,mpqa,roberta-large,30,3,10000.0,4,NaN,NaN,NaN,NaN,NaN,0.7490,0.757143,NaN,NaN
37,none,mpqa,roberta-large,30,4,10000.0,4,NaN,NaN,NaN,NaN,NaN,0.8125,0.795238,NaN,NaN
40,none,mpqa,roberta-large,30,5,10000.0,4,NaN,NaN,NaN,NaN,NaN,0.8780,0.914286,NaN,NaN


mpqa_test_eval_acc 82.52 (5.31)
mpqa-clue_test_eval_acc 83.24 (6.54)


In [84]:
result_list = []
task_list = ['CLUE_prompt_baselines_alltasks_batch128']

args_list = [   'update_component', 'task_name',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
    result_list += get_result_list(task)
result_frame = pd.DataFrame(result_list).drop_duplicates()



360


In [81]:
result_frame

""


In [86]:
for model in ['roberta-large']:
    for task_name in ['qqp', 'rte', 'subj', 'mpqa', 'mnli', 'sst-2']:
        for k in [10, 20, 30]:

            print("shot is {}, epoch is {}, task_name is {} and model name is {}".format(k, epoch, task_name, model))
            display(result_frame[(result_frame['task_name']==task_name)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)])
        
            for  metric in result_frame.columns[result_frame.columns.to_series().str.contains(task_name) & result_frame.columns.to_series().str.contains('test_eval_acc')].to_series():
                
                print(metric, round(result_frame[(result_frame['task_name']==task_name)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)][metric].mean() * 100, 1),  \
                      "("+str(round(result_frame[(result_frame['task_name']==task_name)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)][metric].std()*100,1) )+")")
                

#             print(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].mean()*100, 2),  \
#                   "("+str(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].std()*100, 2))+ ")")

#             print('')

shot is 10, epoch is 50, task_name is qqp and model name is roberta-large


,update_component,task_name,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,mnli_test_eval_mnli/acc,mnli-clue_test_eval_acc,mnli-mm_test_eval_mnli-mm/acc,...,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_acc,rte_test_eval_acc,rte-clue_test_eval_acc,sst-2_test_eval_acc,sst-2-clue_test_eval_acc,mpqa_test_eval_acc,mpqa-clue_test_eval_acc,subj_test_eval_acc,subj-clue_test_eval_acc
105,none,qqp,roberta-large,10,1,50.0,4,NaN,NaN,NaN,...,0.560891,0.657143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,none,qqp,roberta-large,10,2,50.0,4,NaN,NaN,NaN,...,0.493552,0.633333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,none,qqp,roberta-large,10,3,50.0,4,NaN,NaN,NaN,...,0.481515,0.614286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,none,qqp,roberta-large,10,4,50.0,4,NaN,NaN,NaN,...,0.391577,0.538095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,none,qqp,roberta-large,10,5,50.0,4,NaN,NaN,NaN,...,0.551120,0.595238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


qqp_test_eval_acc 57.7 (5.1)
qqp_test_eval_acc_and_f1 49.6 (6.8)
qqp-clue_test_eval_acc 60.8 (4.5)
shot is 20, epoch is 50, task_name is qqp and model name is roberta-large


,update_component,task_name,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,mnli_test_eval_mnli/acc,mnli-clue_test_eval_acc,mnli-mm_test_eval_mnli-mm/acc,...,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_acc,rte_test_eval_acc,rte-clue_test_eval_acc,sst-2_test_eval_acc,sst-2-clue_test_eval_acc,mpqa_test_eval_acc,mpqa-clue_test_eval_acc,subj_test_eval_acc,subj-clue_test_eval_acc
106,none,qqp,roberta-large,20,1,50.0,4,NaN,NaN,NaN,...,0.432474,0.623810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,none,qqp,roberta-large,20,2,50.0,4,NaN,NaN,NaN,...,0.631988,0.671429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,none,qqp,roberta-large,20,3,50.0,4,NaN,NaN,NaN,...,0.495665,0.647619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,none,qqp,roberta-large,20,4,50.0,4,NaN,NaN,NaN,...,0.527815,0.671429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,none,qqp,roberta-large,20,5,50.0,4,NaN,NaN,NaN,...,0.580622,0.604762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


qqp_test_eval_acc 64.9 (3.6)
qqp_test_eval_acc_and_f1 53.4 (7.7)
qqp-clue_test_eval_acc 64.4 (2.9)
shot is 30, epoch is 50, task_name is qqp and model name is roberta-large


...

sst-2_test_eval_acc 89.5 (1.7)
sst-2-clue_test_eval_acc 85.7 (2.9)
shot is 20, epoch is 50, task_name is sst-2 and model name is roberta-large


,update_component,task_name,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,mnli_test_eval_mnli/acc,mnli-clue_test_eval_acc,mnli-mm_test_eval_mnli-mm/acc,...,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_acc,rte_test_eval_acc,rte-clue_test_eval_acc,sst-2_test_eval_acc,sst-2-clue_test_eval_acc,mpqa_test_eval_acc,mpqa-clue_test_eval_acc,subj_test_eval_acc,subj-clue_test_eval_acc
226,none,sst-2,roberta-large,20,1,50.0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.883028,0.895238,NaN,NaN,NaN,NaN
229,none,sst-2,roberta-large,20,2,50.0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.905963,0.876190,NaN,NaN,NaN,NaN
232,none,sst-2,roberta-large,20,3,50.0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.920872,0.866667,NaN,NaN,NaN,NaN
235,none,sst-2,roberta-large,20,4,50.0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.893349,0.861905,NaN,NaN,NaN,NaN
238,none,sst-2,roberta-large,20,5,50.0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.924312,0.880952,NaN,NaN,NaN,NaN


sst-2_test_eval_acc 90.6 (1.8)
sst-2-clue_test_eval_acc 87.6 (1.3)
shot is 30, epoch is 50, task_name is sst-2 and model name is roberta-large


,update_component,task_name,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,mnli_test_eval_mnli/acc,mnli-clue_test_eval_acc,mnli-mm_test_eval_mnli-mm/acc,...,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_acc,rte_test_eval_acc,rte-clue_test_eval_acc,sst-2_test_eval_acc,sst-2-clue_test_eval_acc,mpqa_test_eval_acc,mpqa-clue_test_eval_acc,subj_test_eval_acc,subj-clue_test_eval_acc
227,none,sst-2,roberta-large,30,1,50.0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.901376,0.895238,NaN,NaN,NaN,NaN
230,none,sst-2,roberta-large,30,2,50.0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.922018,0.900000,NaN,NaN,NaN,NaN
233,none,sst-2,roberta-large,30,3,50.0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.923165,0.900000,NaN,NaN,NaN,NaN
236,none,sst-2,roberta-large,30,4,50.0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.922018,0.876190,NaN,NaN,NaN,NaN
239,none,sst-2,roberta-large,30,5,50.0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.907110,0.866667,NaN,NaN,NaN,NaN


sst-2_test_eval_acc 91.5 (1.0)
sst-2-clue_test_eval_acc 88.8 (1.5)


In [27]:

result_list = []
task_list = ['full_finetune_adapter_prompt']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
    result_list += get_result_list(task, 'dev_eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



24


In [36]:
result_frame[(result_frame['adapter_choice'] =='linear_after')&(result_frame['task_name'] =='cola')&(result_frame['model_name_or_path'] =='roberta-large')]

,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,mnli_dev_eval_loss,mnli_dev_eval_mnli/acc,cola_dev_eval_loss,cola_dev_eval_mcc
7,adapter,cola,linear_after,128,roberta-large,-1,1,10.0,32,NaN,NaN,0.426249,0.554667
8,adapter,cola,linear_after,64,roberta-large,-1,1,10.0,32,NaN,NaN,0.476759,0.539699
14,adapter,cola,linear_after,8,roberta-large,-1,1,10.0,32,NaN,NaN,0.517704,0.388289


In [20]:


result_list = []
task_list = ['full_finetune_component_prompt_twotasks']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'dev_eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



16


In [21]:
result_frame

,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,mnli_dev_eval_loss,mnli_dev_eval_mnli/acc,cola_dev_eval_loss,cola_dev_eval_mcc
0,feedforward-output,mnli,none,-1,roberta-base,-1,1,10.0,32,0.410216,0.873051,NaN,NaN
1,attention,mnli,none,-1,roberta-base,-1,1,10.0,32,0.409945,0.875904,NaN,NaN
2,feedforward-intermediate,cola,none,-1,roberta-base,-1,1,10.0,32,NaN,NaN,0.486291,0.568736
3,embedding,mnli,none,-1,roberta-base,-1,1,10.0,32,0.543300,0.784208,NaN,NaN
4,feedforward-intermediate,cola,none,-1,roberta-large,-1,1,10.0,32,NaN,NaN,0.546905,0.643158
5,embedding,cola,none,-1,roberta-base,-1,1,10.0,32,NaN,NaN,0.572096,0.218808
6,feedforward-output,cola,none,-1,roberta-large,-1,1,10.0,32,NaN,NaN,0.990609,0.618200
7,feedforward-output,cola,none,-1,roberta-base,-1,1,10.0,32,NaN,NaN,0.435927,0.588832
8,attention,cola,none,-1,roberta-large,-1,1,10.0,32,NaN,NaN,0.485034,0.648548
9,embedding,cola,none,-1,roberta-large,-1,1,10.0,32,NaN,NaN,0.538619,0.314506


In [22]:
for model in ['roberta-large']:
    for task_name in ['cola']:
        for k in [-1]:

            print("shot is {}, task_name is {} and model name is {}".format(k,  task_name, model))
            display(result_frame[(result_frame['task_name']==task_name)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)])
        
#             for  metric in result_frame.columns[result_frame.columns.to_series().str.contains(task_name) & result_frame.columns.to_series().str.contains('test_eval_acc')].to_series():
                
#                 print(metric, round(result_frame[(result_frame['task_name']==task_name)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)][metric].mean() * 100, 1),  \
#                       "("+str(round(result_frame[(result_frame['task_name']==task_name)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)][metric].std()*100,1) )+")")
                

#             print(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].mean()*100, 2),  \
#                   "("+str(round(result_frame[(result_frame['adapter_init_std']==adapter_init_std)&(result_frame['num_k']==k)&(result_frame['model_name_or_path']==model)]['mnli-mm_test_eval_mnli-mm/acc'].std()*100, 2))+ ")")

#             print('')

shot is -1, task_name is cola and model name is roberta-large


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,mnli_dev_eval_loss,mnli_dev_eval_mnli/acc,cola_dev_eval_loss,cola_dev_eval_mcc
4,feedforward-intermediate,cola,none,-1,roberta-large,-1,1,10.0,32,NaN,NaN,0.546905,0.643158
6,feedforward-output,cola,none,-1,roberta-large,-1,1,10.0,32,NaN,NaN,0.990609,0.618200
8,attention,cola,none,-1,roberta-large,-1,1,10.0,32,NaN,NaN,0.485034,0.648548
9,embedding,cola,none,-1,roberta-large,-1,1,10.0,32,NaN,NaN,0.538619,0.314506


# FULL SUPERVISION

In [75]:

result_list = []
task_list = ['Full_prompt_last_epoch']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



28


In [76]:
result_frame

,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,subj_test_eval_loss,...,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc,rte_test_eval_loss,rte_test_eval_acc,sst-2_test_eval_loss,sst-2_test_eval_acc,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1
0,none,subj,none,-1,roberta-base,-1,1,10.0,32,0.335018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,none,mpqa,none,-1,bert-base-uncased,-1,1,10.0,32,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,none,subj,none,-1,roberta-large,-1,1,10.0,32,0.285739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,none,cola,none,-1,roberta-large,-1,1,10.0,32,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,none,mnli,none,-1,roberta-base,-1,1,10.0,32,NaN,...,1.065804,0.869508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,none,rte,none,-1,bert-base-uncased,-1,1,10.0,32,NaN,...,NaN,NaN,2.211566,0.646209,NaN,NaN,NaN,NaN,NaN,NaN
6,none,sst-2,none,-1,roberta-large,-1,1,10.0,32,NaN,...,NaN,NaN,NaN,NaN,0.430974,0.958716,NaN,NaN,NaN,NaN
7,none,mnli,none,-1,roberta-large,-1,1,10.0,32,NaN,...,1.097115,0.888426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,none,mnli,none,-1,bert-large-uncased,-1,1,10.0,32,NaN,...,1.517821,0.856998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,none,cola,none,-1,bert-base-uncased,-1,1,10.0,32,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:


for model in ['bert-base-uncased', 'bert-large-uncased', 'roberta-base', 'roberta-large']:
    for task_name in ['sst-2', 'subj',  'mpqa',  'mnli', 'rte', 'qqp']:

    


        print(" task_name is {} model is {}".format(task_name, model))
        new_frame = result_frame[(result_frame['model_name_or_path']==model)& (result_frame['task_name']==task_name)].dropna(axis=1)
        display(new_frame)



        for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

            for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                if l in metric:

                    print(metric, round(new_frame[metric]*100, 1))



                    print('')

 task_name is sst-2 model is bert-base-uncased


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,sst-2_test_eval_loss,sst-2_test_eval_acc
21,none,sst-2,none,-1,bert-base-uncased,-1,1,10.0,32,False,64,False,100000,0.970607,0.91055


sst-2_test_eval_acc 21    91.1
Name: sst-2_test_eval_acc, dtype: float64

 task_name is subj model is bert-base-uncased


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,subj_test_eval_loss,subj_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
26,none,subj,none,-1,bert-base-uncased,-1,1,10.0,32,0.466579,0.964,False,64,False,100000


subj_test_eval_acc 26    96.4
Name: subj_test_eval_acc, dtype: float64

 task_name is mpqa model is bert-base-uncased


...

mnli_test_eval_mnli/acc 7    89.3
Name: mnli_test_eval_mnli/acc, dtype: float64

mnli-mm_test_eval_mnli-mm/acc 7    88.8
Name: mnli-mm_test_eval_mnli-mm/acc, dtype: float64

 task_name is rte model is roberta-large


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc
20,none,rte,none,-1,roberta-large,-1,1,10.0,32,False,64,False,100000,1.288235,0.884477


rte_test_eval_acc 20    88.4
Name: rte_test_eval_acc, dtype: float64

 task_name is qqp model is roberta-large


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1
16,none,qqp,none,-1,roberta-large,-1,1,10.0,32,False,64,False,100000,0.646803,0.920604,0.89401,0.907307


qqp_test_eval_acc 16    92.1
Name: qqp_test_eval_acc, dtype: float64

qqp_test_eval_f1 16    89.4
Name: qqp_test_eval_f1, dtype: float64

qqp_test_eval_acc_and_f1 16    90.7
Name: qqp_test_eval_acc_and_f1, dtype: float64

qqp_test_eval_acc_and_f1 16    90.7
Name: qqp_test_eval_acc_and_f1, dtype: float64



In [ ]:
#CLUE_prompt_conventional_FN

# Overall MetaST

In [128]:



result_list = []
task_list = ['CLUE_SEMI_metaST']
task_list = ['CLUE_meatst_gradaccumulat']
args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



30


In [129]:
for task_name in ['sst-2', 'subj',  'mpqa',  'mnli', 'rte', 'qqp']:
    for k in [30]:
       
    


        print(" k is {} task_name is {} ".format(k, task_name))
        new_frame = result_frame[ (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
        display(new_frame)



        for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():
     
            for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                if l in metric:

                    print(metric, round(new_frame[metric].mean() * 100, 1),  \
                          "("+str(round(new_frame[metric].std()*100,1) )+")")



                    print('')

 k is 30 task_name is sst-2 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,sst-2_test_eval_loss,sst-2_test_eval_acc,sst-2-clue_test_eval_loss,sst-2-clue_test_eval_acc
5,none,sst-2,none,-1,roberta-large,30,2,10000.0,4,True,16,False,100,0.628589,0.938073,0.862613,0.909524
11,none,sst-2,none,-1,roberta-large,30,4,10000.0,4,True,16,False,100,0.564380,0.939220,0.645481,0.923810
18,none,sst-2,none,-1,roberta-large,30,5,10000.0,4,True,16,False,100,0.707982,0.920872,0.755128,0.904762
27,none,sst-2,none,-1,roberta-large,30,1,10000.0,4,True,16,False,100,0.688772,0.924312,0.794061,0.909524
28,none,sst-2,none,-1,roberta-large,30,3,10000.0,4,True,16,False,100,0.710630,0.930046,0.765744,0.904762


sst-2_test_eval_acc 93.1 (0.8)

sst-2-clue_test_eval_acc 91.0 (0.8)

 k is 30 task_name is subj 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
0,none,subj,none,-1,roberta-large,30,2,10000.0,4,1.010666,0.9055,1.152688,0.895238,True,16,False,100
3,none,subj,none,-1,roberta-large,30,1,10000.0,4,0.536913,0.9400,0.650531,0.942857,True,16,False,100
6,none,subj,none,-1,roberta-large,30,4,10000.0,4,0.617458,0.9315,0.832405,0.904762,True,16,False,100
7,none,subj,none,-1,roberta-large,30,5,10000.0,4,0.946094,0.9045,1.148260,0.880952,True,16,False,100
21,none,subj,none,-1,roberta-large,30,3,10000.0,4,0.876489,0.9180,1.096255,0.900000,True,16,False,100


subj_test_eval_acc 92.0 (1.6)

subj-clue_test_eval_acc 90.5 (2.3)

 k is 30 task_name is mpqa 


...

mnli_test_eval_mnli/acc 73.9 (3.6)

mnli-clue_test_eval_acc 70.5 (3.2)

mnli-mm_test_eval_mnli-mm/acc 75.0 (3.5)

 k is 30 task_name is rte 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
9,none,rte,none,-1,roberta-large,30,1,10000.0,4,True,16,False,100,2.454360,0.707581,2.727727,0.690476
14,none,rte,none,-1,roberta-large,30,5,10000.0,4,True,16,False,100,2.705002,0.747292,2.640143,0.752381
19,none,rte,none,-1,roberta-large,30,3,10000.0,4,True,16,False,100,3.063040,0.678700,3.212321,0.666667
23,none,rte,none,-1,roberta-large,30,2,10000.0,4,True,16,False,100,3.017394,0.689531,2.761630,0.690476
26,none,rte,none,-1,roberta-large,30,4,10000.0,4,True,16,False,100,3.391793,0.631769,3.194565,0.647619


rte_test_eval_acc 69.1 (4.2)

rte-clue_test_eval_acc 69.0 (3.9)

 k is 30 task_name is qqp 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
8,none,qqp,none,-1,roberta-large,30,3,10000.0,4,True,16,False,100,2.742865,0.732698,0.551148,0.641923,3.306009,0.719048
12,none,qqp,none,-1,roberta-large,30,2,10000.0,4,True,16,False,100,2.402111,0.744051,0.666838,0.705445,2.582554,0.742857
13,none,qqp,none,-1,roberta-large,30,5,10000.0,4,True,16,False,100,4.474255,0.619565,0.593686,0.606626,4.465456,0.633333
17,none,qqp,none,-1,roberta-large,30,4,10000.0,4,True,16,False,100,2.977228,0.732847,0.571644,0.652245,3.794386,0.680952
25,none,qqp,none,-1,roberta-large,30,1,10000.0,4,True,16,False,100,2.799219,0.724610,0.605009,0.664810,2.791288,0.747619


qqp_test_eval_acc 71.1 (5.1)

qqp_test_eval_f1 59.8 (4.4)

qqp_test_eval_acc_and_f1 65.4 (3.6)

qqp_test_eval_acc_and_f1 65.4 (3.6)

qqp-clue_test_eval_acc 70.5 (4.8)



In [181]:



result_list = []
task_list = ['CLUE_SEMI_adapter_metaST']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



48


In [182]:
result_frame

,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,mnli_test_eval_loss,...,sst-2-clue_test_eval_loss,sst-2-clue_test_eval_acc,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc
0,adapter,mnli,lora,128,roberta-large,30,1,10000.0,4,2.479926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,adapter,mnli,lora,128,roberta-large,30,2,10000.0,4,1.859298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,adapter,mnli,lora,128,roberta-large,30,3,10000.0,4,2.180669,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,adapter,mnli,lora,128,roberta-large,30,4,10000.0,4,1.750492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,adapter,mnli,lora,128,roberta-large,30,5,10000.0,4,1.859321,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,adapter,qqp,lora,128,roberta-large,30,1,10000.0,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,adapter,qqp,lora,128,roberta-large,30,2,10000.0,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,adapter,qqp,lora,128,roberta-large,30,3,10000.0,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,adapter,qqp,lora,128,roberta-large,30,4,10000.0,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,adapter,qqp,lora,128,roberta-large,30,5,10000.0,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
for task_name in ['qqp', 'rte', 'subj', 'mpqa', 'mnli', 'sst-2']:
    for k in [30]:
       
    


        print(" k is {} task_name is {} ".format(k, task_name))
        new_frame = result_frame[ (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
        display(new_frame)



        for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():
     
            for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                if l in metric:

                    print(metric, round(new_frame[metric].mean() * 100, 1),  \
                          "("+str(round(new_frame[metric].std()*100,1) )+")")



                    print('')

 k is 30 task_name is qqp 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
5,adapter,qqp,lora,128,roberta-large,30,1,10000.0,4,True,16,False,100,2.289821,0.728321,0.595045,0.661683,2.423407,0.761905
6,adapter,qqp,lora,128,roberta-large,30,2,10000.0,4,True,16,False,100,1.962290,0.748355,0.678628,0.713492,2.044775,0.738095
7,adapter,qqp,lora,128,roberta-large,30,3,10000.0,4,True,16,False,100,2.338986,0.752288,0.592969,0.672628,2.766606,0.747619
8,adapter,qqp,lora,128,roberta-large,30,4,10000.0,4,True,16,False,100,2.220273,0.746574,0.598228,0.672401,2.797331,0.700000
9,adapter,qqp,lora,128,roberta-large,30,5,10000.0,4,True,16,False,100,3.894146,0.614717,0.575848,0.595282,3.652930,0.619048


qqp_test_eval_acc 71.8 (5.8)

qqp_test_eval_f1 60.8 (4.0)

qqp_test_eval_acc_and_f1 66.3 (4.3)

qqp_test_eval_acc_and_f1 66.3 (4.3)

qqp-clue_test_eval_acc 71.3 (5.7)

 k is 30 task_name is rte 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
10,adapter,rte,lora,128,roberta-large,30,1,10000.0,4,True,16,False,100,2.181126,0.649819,2.292499,0.638095
11,adapter,rte,lora,128,roberta-large,30,2,10000.0,4,True,16,False,100,2.062790,0.743682,2.039466,0.752381
12,adapter,rte,lora,128,roberta-large,30,3,10000.0,4,True,16,False,100,2.279244,0.707581,2.410428,0.680952
13,adapter,rte,lora,128,roberta-large,30,4,10000.0,4,True,16,False,100,3.188560,0.617329,2.909858,0.642857
14,adapter,rte,lora,128,roberta-large,30,5,10000.0,4,True,16,False,100,1.980977,0.722022,1.883318,0.714286


rte_test_eval_acc 68.8 (5.3)

rte-clue_test_eval_acc 68.6 (4.8)

 k is 30 task_name is subj 


...

mpqa_test_eval_acc 82.9 (4.0)

mpqa-clue_test_eval_acc 82.5 (4.1)

 k is 30 task_name is mnli 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
0,adapter,mnli,lora,128,roberta-large,30,1,10000.0,4,2.479926,0.628630,2.990610,0.585714,2.488351,0.633238,True,16,False,100
1,adapter,mnli,lora,128,roberta-large,30,2,10000.0,4,1.859298,0.690474,2.004755,0.690476,1.674961,0.716029,True,16,False,100
2,adapter,mnli,lora,128,roberta-large,30,3,10000.0,4,2.180669,0.647580,2.426030,0.633333,2.285801,0.649003,True,16,False,100
3,adapter,mnli,lora,128,roberta-large,30,4,10000.0,4,1.750492,0.707489,2.097489,0.666667,1.647522,0.723454,True,16,False,100
4,adapter,mnli,lora,128,roberta-large,30,5,10000.0,4,1.859321,0.698625,2.137249,0.704762,1.768881,0.717860,True,16,False,100


mnli_test_eval_mnli/acc 67.5 (3.4)

mnli-clue_test_eval_acc 65.6 (4.8)

mnli-mm_test_eval_mnli-mm/acc 68.8 (4.3)

 k is 30 task_name is sst-2 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,sst-2_test_eval_loss,sst-2_test_eval_acc,sst-2-clue_test_eval_loss,sst-2-clue_test_eval_acc
15,adapter,sst-2,lora,128,roberta-large,30,1,10000.0,4,True,16,False,100,0.754129,0.909404,0.855620,0.890476
16,adapter,sst-2,lora,128,roberta-large,30,2,10000.0,4,True,16,False,100,0.695019,0.935780,0.960666,0.890476
17,adapter,sst-2,lora,128,roberta-large,30,3,10000.0,4,True,16,False,100,1.146369,0.891055,0.771655,0.890476
18,adapter,sst-2,lora,128,roberta-large,30,4,10000.0,4,True,16,False,100,0.711769,0.922018,0.851372,0.885714
19,adapter,sst-2,lora,128,roberta-large,30,5,10000.0,4,True,16,False,100,0.766369,0.918578,0.598317,0.900000


sst-2_test_eval_acc 91.5 (1.7)

sst-2-clue_test_eval_acc 89.1 (0.5)



In [121]:





result_list = []
task_list = ['CLUE_meatst_gradaccumulat_soft']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



30


In [122]:
for task_name in [ 'mnli', 'rte', 'qqp', 'sst-2', 'subj',  'mpqa']:
    for k in [30]:
       
    


        print(" k is {} task_name is {} ".format(k, task_name))
        new_frame = result_frame[ (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
        display(new_frame)



        for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():
     
            for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                if l in metric:

                    print(metric, round(new_frame[metric].mean() * 100, 1),  \
                          "("+str(round(new_frame[metric].std()*100,1) )+")")



                    print('')

 k is 30 task_name is mnli 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
0,none,mnli,none,-1,roberta-large,30,1,10000.0,4,2.045647,0.701274,2.504867,0.661905,1.947423,0.712469,True,16,False,100
1,none,mnli,none,-1,roberta-large,30,2,10000.0,4,1.544016,0.771167,2.035275,0.709524,1.433443,0.780207,True,16,False,100
2,none,mnli,none,-1,roberta-large,30,3,10000.0,4,1.602074,0.738054,1.860211,0.695238,1.626093,0.734947,True,16,False,100
3,none,mnli,none,-1,roberta-large,30,4,10000.0,4,1.677991,0.749873,2.186749,0.723810,1.567171,0.768714,True,16,False,100
4,none,mnli,none,-1,roberta-large,30,5,10000.0,4,1.949956,0.719307,2.360058,0.709524,1.787716,0.742270,True,16,False,100


mnli_test_eval_mnli/acc 73.6 (2.7)

mnli-clue_test_eval_acc 70.0 (2.4)

mnli-mm_test_eval_mnli-mm/acc 74.8 (2.7)

 k is 30 task_name is rte 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
9,none,rte,none,-1,roberta-large,30,1,10000.0,4,True,16,False,100,2.078779,0.675090,2.181484,0.661905
10,none,rte,none,-1,roberta-large,30,2,10000.0,4,True,16,False,100,1.998994,0.725632,1.795124,0.738095
11,none,rte,none,-1,roberta-large,30,3,10000.0,4,True,16,False,100,2.689237,0.700361,2.801738,0.685714


rte_test_eval_acc 70.0 (2.5)

rte-clue_test_eval_acc 69.5 (3.9)

 k is 30 task_name is qqp 


...

sst-2_test_eval_acc 92.5 (0.6)

sst-2-clue_test_eval_acc 91.0 (1.0)

 k is 30 task_name is subj 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc
21,none,subj,none,-1,roberta-large,30,1,10000.0,4,True,16,False,100,0.599454,0.9290,0.694287,0.933333
22,none,subj,none,-1,roberta-large,30,2,10000.0,4,True,16,False,100,0.652629,0.9200,0.708871,0.909524
23,none,subj,none,-1,roberta-large,30,3,10000.0,4,True,16,False,100,0.675247,0.9185,0.800397,0.914286
24,none,subj,none,-1,roberta-large,30,4,10000.0,4,True,16,False,100,0.557351,0.9255,0.745073,0.914286
25,none,subj,none,-1,roberta-large,30,5,10000.0,4,True,16,False,100,0.563291,0.9280,0.573496,0.923810


subj_test_eval_acc 92.4 (0.5)

subj-clue_test_eval_acc 91.9 (1.0)

 k is 30 task_name is mpqa 


,update_component,task_name,adapter_choice,adapter_dim,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc
16,none,mpqa,none,-1,roberta-large,30,1,10000.0,4,True,16,False,100,1.277639,0.8555,1.278908,0.861905
17,none,mpqa,none,-1,roberta-large,30,2,10000.0,4,True,16,False,100,1.957829,0.7935,1.795313,0.819048
18,none,mpqa,none,-1,roberta-large,30,3,10000.0,4,True,16,False,100,1.791493,0.8075,1.731101,0.809524
19,none,mpqa,none,-1,roberta-large,30,4,10000.0,4,True,16,False,100,1.376820,0.8535,1.465377,0.838095
20,none,mpqa,none,-1,roberta-large,30,5,10000.0,4,True,16,False,100,1.397735,0.8560,1.583303,0.842857


mpqa_test_eval_acc 83.3 (3.0)

mpqa-clue_test_eval_acc 83.4 (2.1)



In [275]:






result_list = []
task_list = ['CLUE_metast_newtasks']

args_list = [  'update_component', 'task_name', 'self_training_session', 'adapter_choice', 'adapter_dim', 'soft_label',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



30


In [276]:

for task_name in [ 'rte', 'qqp',  'mpqa']:
    for k in [30]:
        for soft_label in [1, 0]:
       
    


            print(" k is {} task_name is {} soft_label is {} ".format(k, task_name, soft_label))
            new_frame = result_frame[(result_frame['soft_label']==soft_label)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 30 task_name is rte soft_label is 1 


,update_component,task_name,self_training_session,adapter_choice,adapter_dim,soft_label,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
0,none,rte,6,none,-1,1,roberta-large,30,3,10000.0,4,2.003414,0.732852,2.042239,0.733333,True,16,False,100
1,none,rte,6,none,-1,1,roberta-large,30,5,10000.0,4,1.949032,0.743682,1.971717,0.733333,True,16,False,100
3,none,rte,6,none,-1,1,roberta-large,30,2,10000.0,4,1.752773,0.696751,1.629660,0.695238,True,16,False,100
8,none,rte,6,none,-1,1,roberta-large,30,4,10000.0,4,2.252921,0.729242,2.309651,0.714286,True,16,False,100
19,none,rte,6,none,-1,1,roberta-large,30,1,10000.0,4,1.714007,0.678700,1.851017,0.676190,True,16,False,100


rte_test_eval_acc 71.6 (2.7)

rte-clue_test_eval_acc 71.0 (2.5)

 k is 30 task_name is rte soft_label is 0 


,update_component,task_name,self_training_session,adapter_choice,adapter_dim,soft_label,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
5,none,rte,6,none,-1,0,roberta-large,30,4,10000.0,4,3.362934,0.703971,3.563308,0.695238,True,16,False,100
10,none,rte,6,none,-1,0,roberta-large,30,3,10000.0,4,2.740285,0.667870,2.780880,0.657143,True,16,False,100
13,none,rte,6,none,-1,0,roberta-large,30,5,10000.0,4,2.246007,0.743682,2.164056,0.738095,True,16,False,100
15,none,rte,6,none,-1,0,roberta-large,30,2,10000.0,4,2.660066,0.714801,2.360924,0.733333,True,16,False,100
21,none,rte,6,none,-1,0,roberta-large,30,1,10000.0,4,2.465513,0.667870,2.662958,0.652381,True,16,False,100


rte_test_eval_acc 70.0 (3.2)

rte-clue_test_eval_acc 69.5 (4.1)

 k is 30 task_name is qqp soft_label is 1 


...

rte_test_eval_acc nan (nan)

rte-clue_test_eval_acc nan (nan)

eval_accumulation_steps nan (nan)

mpqa_test_eval_acc nan (nan)

mpqa-clue_test_eval_acc nan (nan)

 k is 30 task_name is mpqa soft_label is 1 


,update_component,task_name,self_training_session,adapter_choice,adapter_dim,soft_label,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc
2,none,mpqa,6,none,-1,1,roberta-large,30,1,10000.0,4,True,16,False,100,0.891342,0.8525,0.898035,0.857143
4,none,mpqa,6,none,-1,1,roberta-large,30,4,10000.0,4,True,16,False,100,1.232911,0.8495,1.204220,0.842857
7,none,mpqa,6,none,-1,1,roberta-large,30,3,10000.0,4,True,16,False,100,0.925216,0.8820,0.841700,0.880952
18,none,mpqa,6,none,-1,1,roberta-large,30,5,10000.0,4,True,16,False,100,1.043218,0.8735,0.997953,0.880952
23,none,mpqa,6,none,-1,1,roberta-large,30,2,10000.0,4,True,16,False,100,1.100589,0.8560,1.299576,0.823810


mpqa_test_eval_acc 86.3 (1.4)

mpqa-clue_test_eval_acc 85.7 (2.5)

 k is 30 task_name is mpqa soft_label is 0 


,update_component,task_name,self_training_session,adapter_choice,adapter_dim,soft_label,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc
6,none,mpqa,6,none,-1,0,roberta-large,30,2,10000.0,4,True,16,False,100,1.275690,0.8465,1.463380,0.819048
12,none,mpqa,6,none,-1,0,roberta-large,30,3,10000.0,4,True,16,False,100,0.882318,0.8895,0.731140,0.904762
16,none,mpqa,6,none,-1,0,roberta-large,30,4,10000.0,4,True,16,False,100,1.208094,0.8455,1.207136,0.828571
24,none,mpqa,6,none,-1,0,roberta-large,30,5,10000.0,4,True,16,False,100,1.182416,0.8585,1.021153,0.871429
26,none,mpqa,6,none,-1,0,roberta-large,30,1,10000.0,4,True,16,False,100,0.984539,0.8460,0.975667,0.852381


mpqa_test_eval_acc 85.7 (1.9)

mpqa-clue_test_eval_acc 85.5 (3.4)



In [198]:



result_list = []
task_list = ['CLUE_prompt_metast_warumup']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'soft_label',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()


60


In [193]:
result_frame

,update_component,task_name,adapter_choice,adapter_dim,soft_label,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,...,sst-2-clue_test_eval_loss,sst-2-clue_test_eval_acc,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc
0,none,mnli,none,-1,1,roberta-large,30,1,10000.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,none,mnli,none,-1,0,roberta-large,30,1,10000.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,none,mnli,none,-1,1,roberta-large,30,2,10000.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,none,mnli,none,-1,1,roberta-large,30,3,10000.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,none,qqp,none,-1,1,roberta-large,30,1,10000.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,none,qqp,none,-1,0,roberta-large,30,3,10000.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,none,qqp,none,-1,1,roberta-large,30,4,10000.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,none,qqp,none,-1,0,roberta-large,30,5,10000.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,none,rte,none,-1,0,roberta-large,30,1,10000.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,none,rte,none,-1,0,roberta-large,30,2,10000.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [274]:

for task_name in ['sst-2', 'subj',  'mpqa',  'mnli', 'rte', 'qqp']:
    for k in [30]:
        for soft_label in [1]:
       
    


            print(" k is {} task_name is {} soft_label is {} ".format(k, task_name, soft_label))
            new_frame = result_frame[(result_frame['soft_label']==soft_label)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 30 task_name is sst-2 soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,sst-2_test_eval_loss,sst-2_test_eval_acc,sst-2-clue_test_eval_loss,sst-2-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
0,0.0,0.0001,adapter,sst-2,linear_after,128,0.000005,128,1,roberta-large,30,5,0.447017,0.917431,0.444609,0.928571,True,16,False,100
1,0.0,0.0001,adapter,sst-2,linear_after,128,0.000005,128,1,roberta-large,30,2,0.437925,0.930046,0.515770,0.914286,True,16,False,100
2,0.0,0.0001,adapter,sst-2,linear_after,128,0.000005,128,1,roberta-large,30,3,0.486988,0.920872,0.435575,0.928571,True,16,False,100
5,0.0,0.0001,adapter,sst-2,linear_after,128,0.000005,128,1,roberta-large,30,1,0.461741,0.931193,0.500468,0.919048,True,16,False,100
10,0.0,0.0001,adapter,sst-2,linear_after,128,0.000005,128,1,roberta-large,30,4,0.428949,0.931193,0.508880,0.909524,True,16,False,100


sst-2_test_eval_acc 92.6 (0.7)

sst-2-clue_test_eval_acc 92.0 (0.9)

 k is 30 task_name is subj soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc
3,0.0,0.0001,adapter,subj,linear_after,128,0.000005,128,1,roberta-large,30,1,True,16,False,100,0.314061,0.955,0.449260,0.942857
12,0.0,0.0001,adapter,subj,linear_after,128,0.000005,128,1,roberta-large,30,3,True,16,False,100,0.422344,0.929,0.512213,0.919048
27,0.0,0.0001,adapter,subj,linear_after,128,0.000005,128,1,roberta-large,30,2,True,16,False,100,0.507917,0.924,0.539930,0.919048
28,0.0,0.0001,adapter,subj,linear_after,128,0.000005,128,1,roberta-large,30,4,True,16,False,100,0.349531,0.942,0.507601,0.909524
29,0.0,0.0001,adapter,subj,linear_after,128,0.000005,128,1,roberta-large,30,5,True,16,False,100,0.470333,0.928,0.487829,0.919048


subj_test_eval_acc 93.6 (1.3)

subj-clue_test_eval_acc 92.2 (1.2)

 k is 30 task_name is mpqa soft_label is 1 


...

mnli_test_eval_mnli/acc 67.9 (12.9)

mnli-clue_test_eval_acc 63.5 (14.5)

mnli-mm_test_eval_mnli-mm/acc 69.0 (13.1)

 k is 30 task_name is rte soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
7,0.0,0.0001,adapter,rte,linear_after,128,0.000005,128,1,roberta-large,30,5,True,16,False,100,0.648388,0.613718,0.654955,0.604762
9,0.0,0.0001,adapter,rte,linear_after,128,0.000005,128,1,roberta-large,30,4,True,16,False,100,1.852066,0.725632,1.959992,0.709524
13,0.0,0.0001,adapter,rte,linear_after,128,0.000005,128,1,roberta-large,30,2,True,16,False,100,1.665587,0.718412,1.635177,0.709524
16,0.0,0.0001,adapter,rte,linear_after,128,0.000005,128,1,roberta-large,30,1,True,16,False,100,1.787750,0.696751,1.879068,0.690476
22,0.0,0.0001,adapter,rte,linear_after,128,0.000005,128,1,roberta-large,30,3,True,16,False,100,1.913469,0.703971,1.942616,0.700000


rte_test_eval_acc 69.2 (4.5)

rte-clue_test_eval_acc 68.3 (4.4)

 k is 30 task_name is qqp soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
8,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.840890,0.750754,0.591809,0.671282,2.117408,0.738095
15,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.710922,0.749567,0.671703,0.710635,1.534919,0.766667
19,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.796145,0.739970,0.588742,0.664356,1.922073,0.757143
20,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.487379,0.693767,0.535997,0.614882,1.577455,0.685714
21,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.448814,0.763789,0.696170,0.729979,1.563080,0.757143


qqp_test_eval_acc 74.0 (2.7)

qqp_test_eval_f1 61.7 (6.6)

qqp_test_eval_acc_and_f1 67.8 (4.5)

qqp_test_eval_acc_and_f1 67.8 (4.5)

qqp-clue_test_eval_acc 74.1 (3.3)



In [210]:



result_list = []
task_list = ['CLUE_prompt_metast_nowarmup']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'soft_label', 'un_gradient_accumulation_steps',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()


60


In [213]:

for task_name in [ 'mnli', 'rte', 'qqp']:
    for k in [30]:
        for soft_label in [1, 0]:
            for un_gradient_accumulation_steps in [1]:
       
    


                print(" k is {} task_name is {} soft_label is {} un_gradient_accumulation_steps is {}".format(k, task_name, soft_label, un_gradient_accumulation_steps))
                new_frame = result_frame[(result_frame['un_gradient_accumulation_steps']==un_gradient_accumulation_steps)&(result_frame['soft_label']==soft_label)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
                display(new_frame)



                for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                    for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                        if l in metric:

                            print(metric, round(new_frame[metric].mean() * 100, 1),  \
                                  "("+str(round(new_frame[metric].std()*100,1) )+")")



                            print('')

 k is 30 task_name is mnli soft_label is 1 un_gradient_accumulation_steps is 1


,update_component,task_name,adapter_choice,adapter_dim,soft_label,un_gradient_accumulation_steps,model_name_or_path,num_k,seed,num_train_epochs,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc
3,none,mnli,none,-1,1,1,roberta-large,30,5,10000.0,...,True,16,False,100,1.770974,0.720530,2.093475,0.709524,1.665821,0.736473
12,none,mnli,none,-1,1,1,roberta-large,30,3,10000.0,...,True,16,False,100,1.668759,0.724401,1.954112,0.714286,1.704422,0.718775
51,none,mnli,none,-1,1,1,roberta-large,30,4,10000.0,...,True,16,False,100,1.566706,0.741416,2.003084,0.733333,1.447420,0.758442
58,none,mnli,none,-1,1,1,roberta-large,30,2,10000.0,...,True,16,False,100,1.212242,0.771167,1.475086,0.733333,1.054190,0.792413
59,none,mnli,none,-1,1,1,roberta-large,30,1,10000.0,...,True,16,False,100,2.344554,0.669078,3.013494,0.600000,2.243765,0.677990


mnli_test_eval_mnli/acc 72.5 (3.7)

mnli-clue_test_eval_acc 69.8 (5.6)

mnli-mm_test_eval_mnli-mm/acc 73.7 (4.3)

 k is 30 task_name is mnli soft_label is 0 un_gradient_accumulation_steps is 1


,update_component,task_name,adapter_choice,adapter_dim,soft_label,un_gradient_accumulation_steps,model_name_or_path,num_k,seed,num_train_epochs,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc
9,none,mnli,none,-1,0,1,roberta-large,30,5,10000.0,...,True,16,False,100,2.048022,0.711564,2.380585,0.719048,1.937530,0.726810
10,none,mnli,none,-1,0,1,roberta-large,30,1,10000.0,...,True,16,False,100,2.455381,0.660723,2.966983,0.623810,2.253475,0.675244
24,none,mnli,none,-1,0,1,roberta-large,30,2,10000.0,...,True,16,False,100,1.536897,0.770250,1.658651,0.723810,1.426889,0.783259
48,none,mnli,none,-1,0,1,roberta-large,30,4,10000.0,...,True,16,False,100,1.795991,0.746409,2.375319,0.700000,1.726335,0.754882
53,none,mnli,none,-1,0,1,roberta-large,30,3,10000.0,...,True,16,False,100,1.823597,0.725522,2.097310,0.676190,1.864570,0.720098


mnli_test_eval_mnli/acc 72.3 (4.1)

mnli-clue_test_eval_acc 68.9 (4.1)

mnli-mm_test_eval_mnli-mm/acc 73.2 (4.0)

 k is 30 task_name is rte soft_label is 1 un_gradient_accumulation_steps is 1


...

rte_test_eval_acc 69.5 (3.5)

rte-clue_test_eval_acc 69.4 (3.7)

 k is 30 task_name is qqp soft_label is 1 un_gradient_accumulation_steps is 1


,update_component,task_name,adapter_choice,adapter_dim,soft_label,un_gradient_accumulation_steps,model_name_or_path,num_k,seed,num_train_epochs,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
30,none,qqp,none,-1,1,1,roberta-large,30,1,10000.0,...,True,16,False,100,2.221302,0.743359,0.635162,0.689260,2.099062,0.766667
35,none,qqp,none,-1,1,1,roberta-large,30,2,10000.0,...,True,16,False,100,2.102834,0.737942,0.658369,0.698156,2.023014,0.757143
38,none,qqp,none,-1,1,1,roberta-large,30,4,10000.0,...,True,16,False,100,2.480271,0.712886,0.572386,0.642636,2.796300,0.690476
52,none,qqp,none,-1,1,1,roberta-large,30,5,10000.0,...,True,16,False,100,2.101916,0.761538,0.687599,0.724569,2.127537,0.757143
57,none,qqp,none,-1,1,1,roberta-large,30,3,10000.0,...,True,16,False,100,2.417588,0.738585,0.570348,0.654467,2.823436,0.700000


qqp_test_eval_acc 73.9 (1.7)

qqp_test_eval_f1 62.5 (5.2)

qqp_test_eval_acc_and_f1 68.2 (3.3)

qqp_test_eval_acc_and_f1 68.2 (3.3)

qqp-clue_test_eval_acc 73.4 (3.6)

 k is 30 task_name is qqp soft_label is 0 un_gradient_accumulation_steps is 1


,update_component,task_name,adapter_choice,adapter_dim,soft_label,un_gradient_accumulation_steps,model_name_or_path,num_k,seed,num_train_epochs,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
7,none,qqp,none,-1,0,1,roberta-large,30,2,10000.0,...,True,16,False,100,2.386728,0.751546,0.671528,0.711537,2.447781,0.738095
13,none,qqp,none,-1,0,1,roberta-large,30,5,10000.0,...,True,16,False,100,2.206469,0.756468,0.688556,0.722512,2.069227,0.780952
21,none,qqp,none,-1,0,1,roberta-large,30,3,10000.0,...,True,16,False,100,2.712480,0.727603,0.533723,0.630663,3.056266,0.723810
26,none,qqp,none,-1,0,1,roberta-large,30,4,10000.0,...,True,16,False,100,2.754011,0.720010,0.579245,0.649627,3.170644,0.680952
44,none,qqp,none,-1,0,1,roberta-large,30,1,10000.0,...,True,16,False,100,2.363752,0.737472,0.614905,0.676188,2.076866,0.780952


qqp_test_eval_acc 73.9 (1.5)

qqp_test_eval_f1 61.8 (6.4)

qqp_test_eval_acc_and_f1 67.8 (3.9)

qqp_test_eval_acc_and_f1 67.8 (3.9)

qqp-clue_test_eval_acc 74.1 (4.2)



# FEW-SHOT BASELINES

In [123]:





result_list = []
task_list = ['CLUE_prompt_baselines_alltasks_batch128']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'max_seq_length',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



360


In [124]:

for k in [10, 20, 30]:
    for model in ['bert-base-uncased', 'bert-large-uncased', 'roberta-base', 'roberta-large']:
        for task_name in ['sst-2', 'subj',  'mpqa',  'mnli', 'rte', 'qqp']:
       
    


            print(" k is {} task_name is {} model is {}".format(k, task_name, model))
            new_frame = result_frame[(result_frame['model_name_or_path']==model)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 10 task_name is sst-2 model is bert-base-uncased


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,sst-2_test_eval_loss,sst-2_test_eval_acc,sst-2-clue_test_eval_loss,sst-2-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
0,none,sst-2,none,-1,128,bert-base-uncased,10,5,50.0,4,1.156706,0.720183,1.318395,0.609524,True,16,False,50
128,none,sst-2,none,-1,128,bert-base-uncased,10,4,50.0,4,1.910080,0.616972,1.113072,0.709524,True,16,False,50
197,none,sst-2,none,-1,128,bert-base-uncased,10,3,50.0,4,0.648794,0.775229,0.625448,0.790476,True,16,False,50
206,none,sst-2,none,-1,128,bert-base-uncased,10,1,50.0,4,0.728900,0.766055,0.708746,0.728571,True,16,False,50
220,none,sst-2,none,-1,128,bert-base-uncased,10,2,50.0,4,0.995410,0.665138,0.727513,0.776190,True,16,False,50


sst-2_test_eval_acc 70.9 (6.7)

sst-2-clue_test_eval_acc 72.3 (7.2)

 k is 10 task_name is subj model is bert-base-uncased


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc
34,none,subj,none,-1,128,bert-base-uncased,10,4,50.0,4,True,16,False,50,0.647351,0.8235,0.561783,0.828571
90,none,subj,none,-1,128,bert-base-uncased,10,2,50.0,4,True,16,False,50,0.598302,0.8150,0.755364,0.780952
236,none,subj,none,-1,128,bert-base-uncased,10,1,50.0,4,True,16,False,50,0.743166,0.8015,0.878672,0.757143
330,none,subj,none,-1,128,bert-base-uncased,10,3,50.0,4,True,16,False,50,0.720724,0.8140,0.681664,0.814286
358,none,subj,none,-1,128,bert-base-uncased,10,5,50.0,4,True,16,False,50,2.234811,0.5790,2.206800,0.566667


subj_test_eval_acc 76.7 (10.5)

subj-clue_test_eval_acc 75.0 (10.6)

 k is 10 task_name is mpqa model is bert-base-uncased


...

mnli_test_eval_mnli/acc 62.8 (2.6)

mnli-clue_test_eval_acc 59.7 (2.8)

mnli-mm_test_eval_mnli-mm/acc 64.1 (3.3)

 k is 30 task_name is rte model is roberta-large


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
4,none,rte,none,-1,128,roberta-large,30,4,50.0,4,True,16,False,50,1.760255,0.599278,1.593762,0.619048
5,none,rte,none,-1,128,roberta-large,30,2,50.0,4,True,16,False,50,1.795058,0.657040,1.705341,0.671429
161,none,rte,none,-1,128,roberta-large,30,3,50.0,4,True,16,False,50,2.030255,0.638989,2.018767,0.638095
164,none,rte,none,-1,128,roberta-large,30,1,50.0,4,True,16,False,50,1.583848,0.642599,1.608818,0.642857
225,none,rte,none,-1,128,roberta-large,30,5,50.0,4,True,16,False,50,1.827574,0.685921,1.896949,0.666667


rte_test_eval_acc 64.5 (3.1)

rte-clue_test_eval_acc 64.8 (2.2)

 k is 30 task_name is qqp model is roberta-large


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
83,none,qqp,none,-1,128,roberta-large,30,1,50.0,4,True,16,False,50,2.090154,0.730176,0.600856,0.665516,2.587018,0.719048
215,none,qqp,none,-1,128,roberta-large,30,4,50.0,4,True,16,False,50,1.813343,0.703116,0.588276,0.645696,2.154517,0.666667
226,none,qqp,none,-1,128,roberta-large,30,2,50.0,4,True,16,False,50,2.011345,0.711056,0.596142,0.653599,2.099030,0.704762
240,none,qqp,none,-1,128,roberta-large,30,3,50.0,4,True,16,False,50,3.462692,0.689364,0.421644,0.555504,3.758532,0.680952
280,none,qqp,none,-1,128,roberta-large,30,5,50.0,4,True,16,False,50,3.504530,0.616522,0.575582,0.596052,3.143696,0.647619


qqp_test_eval_acc 69.0 (4.4)

qqp_test_eval_f1 55.6 (7.6)

qqp_test_eval_acc_and_f1 62.3 (4.6)

qqp_test_eval_acc_and_f1 62.3 (4.6)

qqp-clue_test_eval_acc 68.4 (2.9)



In [154]:






result_list = []
task_list = ['CLUE_prompt_extra_new_tasks']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'max_seq_length',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



313


In [155]:

for k in [10, 20, 30]:
    for model in [ 't5-small', 't5-base', 't5-large', 'roberta-large']:
        for task_name in  ['rte', 'qqp', 'mpqa']:
       
    


            print(" k is {} task_name is {} model is {}".format(k, task_name, model))
            new_frame = result_frame[(result_frame['model_name_or_path']==model)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 10 task_name is rte model is t5-small


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
51,none,rte,none,-1,128,t5-small,10,3,50.0,4,0.772536,0.537906,0.767275,0.528571,True,16,False,50
75,none,rte,none,-1,128,t5-small,10,5,50.0,4,0.867807,0.523466,0.859360,0.519048,True,16,False,50
113,none,rte,none,-1,128,t5-small,10,2,50.0,4,0.749880,0.566787,0.736173,0.585714,True,16,False,50
174,none,rte,none,-1,128,t5-small,10,4,50.0,4,0.840811,0.537906,0.829757,0.533333,True,16,False,50
224,none,rte,none,-1,128,t5-small,10,1,50.0,4,0.807940,0.541516,0.787716,0.566667,True,16,False,50


rte_test_eval_acc 54.2 (1.6)

rte-clue_test_eval_acc 54.7 (2.8)

 k is 10 task_name is qqp model is t5-small


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
9,none,qqp,none,-1,128,t5-small,10,5,50.0,4,True,16,False,50,0.812271,0.441751,0.483619,0.462685,0.818279,0.452381
136,none,qqp,none,-1,128,t5-small,10,4,50.0,4,True,16,False,50,0.747666,0.529805,0.342669,0.436237,0.755210,0.538095
228,none,qqp,none,-1,128,t5-small,10,1,50.0,4,True,16,False,50,0.724534,0.563196,0.318725,0.440961,0.723813,0.566667
283,none,qqp,none,-1,128,t5-small,10,2,50.0,4,True,16,False,50,0.775132,0.484789,0.430812,0.457800,0.783279,0.490476
311,none,qqp,none,-1,128,t5-small,10,3,50.0,4,True,16,False,50,0.808914,0.439847,0.476987,0.458417,0.825440,0.438095


qqp_test_eval_acc 49.2 (5.4)

qqp_test_eval_f1 41.1 (7.6)

qqp_test_eval_acc_and_f1 45.1 (1.2)

qqp_test_eval_acc_and_f1 45.1 (1.2)

qqp-clue_test_eval_acc 49.7 (5.5)

 k is 10 task_name is mpqa model is t5-small


...

rte_test_eval_acc 66.1 (2.2)

rte-clue_test_eval_acc 65.8 (1.6)

 k is 30 task_name is qqp model is roberta-large


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
125,none,qqp,none,-1,128,roberta-large,30,2,50.0,4,True,16,False,50,2.011345,0.711056,0.596142,0.653599,2.099030,0.704762
169,none,qqp,none,-1,128,roberta-large,30,5,50.0,4,True,16,False,50,2.346138,0.718996,0.670972,0.694984,2.737380,0.709524
202,none,qqp,none,-1,128,roberta-large,30,1,50.0,4,True,16,False,50,2.090154,0.730176,0.600856,0.665516,2.587018,0.719048
246,none,qqp,none,-1,128,roberta-large,30,4,50.0,4,True,16,False,50,1.813343,0.703116,0.588276,0.645696,2.154517,0.666667
262,none,qqp,none,-1,128,roberta-large,30,3,50.0,4,True,16,False,50,3.462692,0.689364,0.421644,0.555504,3.758532,0.680952


qqp_test_eval_acc 71.1 (1.5)

qqp_test_eval_f1 57.6 (9.2)

qqp_test_eval_acc_and_f1 64.3 (5.2)

qqp_test_eval_acc_and_f1 64.3 (5.2)

qqp-clue_test_eval_acc 69.6 (2.2)

 k is 30 task_name is mpqa model is roberta-large


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc
57,none,mpqa,none,-1,128,roberta-large,30,3,50.0,4,True,16,False,50,1.416894,0.8565,1.309883,0.871429
172,none,mpqa,none,-1,128,roberta-large,30,4,50.0,4,True,16,False,50,1.496768,0.8405,1.646639,0.819048
198,none,mpqa,none,-1,128,roberta-large,30,2,50.0,4,True,16,False,50,3.203775,0.7610,3.141963,0.761905
213,none,mpqa,none,-1,128,roberta-large,30,1,50.0,4,True,16,False,50,1.616945,0.8270,1.545312,0.819048
251,none,mpqa,none,-1,128,roberta-large,30,5,50.0,4,True,16,False,50,1.426271,0.8490,1.501555,0.847619


mpqa_test_eval_acc 82.7 (3.8)

mpqa-clue_test_eval_acc 82.4 (4.1)



In [ ]:
mpqa seed -2
mpqa seed -3
rte - 4
qqp-5

# ZERO-SHOT

In [89]:





result_list = []
task_list = ['CLUE_prompt_zero_shot_once']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'max_seq_length',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



114


In [ ]:
result_frame

In [90]:

for k in [10, 20, 30]:
    for model in ['bert-base-uncased', 'bert-large-uncased', 'roberta-base', 'roberta-large']:
        for task_name in ['sst-2', 'subj',  'mpqa',  'mnli', 'rte', 'qqp']:
       
    


            print(" k is {} task_name is {} model is {}".format(k, task_name, model))
            new_frame = result_frame[(result_frame['model_name_or_path']==model)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 10 task_name is sst-2 model is bert-base-uncased


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,sst-2_test_eval_loss,sst-2_test_eval_acc,sst-2-clue_test_eval_loss,sst-2-clue_test_eval_acc
29,none,sst-2,none,-1,128,bert-base-uncased,10,1,50.0,4,True,16,False,50,0.695755,0.615826,0.568881,0.719048


sst-2_test_eval_acc 61.6 (nan)

sst-2-clue_test_eval_acc 71.9 (nan)

 k is 10 task_name is subj model is bert-base-uncased


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc
54,none,subj,none,-1,128,bert-base-uncased,10,1,50.0,4,True,16,False,50,0.664569,0.593,0.666037,0.585714


subj_test_eval_acc 59.3 (nan)

subj-clue_test_eval_acc 58.6 (nan)

 k is 10 task_name is mpqa model is bert-base-uncased


...

mnli_test_eval_mnli/acc 51.7 (nan)

mnli-clue_test_eval_acc 49.0 (nan)

mnli-mm_test_eval_mnli-mm/acc 52.4 (nan)

 k is 30 task_name is rte model is roberta-large


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
14,none,rte,none,-1,128,roberta-large,30,1,50.0,4,True,16,False,50,1.310545,0.512635,1.34042,0.504762


rte_test_eval_acc 51.3 (nan)

rte-clue_test_eval_acc 50.5 (nan)

 k is 30 task_name is qqp model is roberta-large


,update_component,task_name,adapter_choice,adapter_dim,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
90,none,qqp,none,-1,128,roberta-large,30,1,50.0,4,1.259353,0.386075,0.497215,0.441645,1.282475,0.380952,True,16,False,50


qqp_test_eval_acc 38.6 (nan)

qqp_test_eval_f1 49.7 (nan)

qqp_test_eval_acc_and_f1 44.2 (nan)

qqp_test_eval_acc_and_f1 44.2 (nan)

qqp-clue_test_eval_acc 38.1 (nan)



In [ ]:
# CLUE_finetune_component_prompt_all

In [112]:


result_list = []
task_list = ['CLUE_finetune_component_prompt_all']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



360


In [114]:
result_frame.columns

Index([], dtype='object')

In [113]:
for update_component in ['embedding', 'feedforward-intermediate', 'feedforward-output', 'attention']:
   
        
    for k in [10, 20, 30]:


        for task_name in ['sst-2', 'subj',  'mpqa',  'mnli', 'rte', 'qqp']:

    


            print(" k is {} update_component is {} model name is {} task_name is {} ".format(k, update_component, model, task_name))
            new_frame = result_frame[(result_frame['update_component']==update_component)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 10 update_component is embedding model name is t5-large task_name is sst-2 


KeyError: 'update_component'

# T5

In [105]:





result_list = []
task_list = ['CLUE_prompt_T5_4checkpoint', 'CLUE_prompt_T5_3checkpoint']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



279
540


In [98]:
result_frame['learning_rate']

0      0.000005
1      0.000005
2      0.000005
3      0.000005
4      0.000005
         ...   
762    0.001000
763    0.001000
764    0.001000
765    0.001000
766    0.000100
Name: learning_rate, Length: 767, dtype: float64

In [106]:
for model in ['t5-small', 't5-base', 't5-large']:
    for lr in [1e-3, 1e-4, 5e-6]:

    
        
        for k in [ 30]:
        
        
            for task_name in ['sst-2', 'subj',  'mpqa',  'mnli', 'rte', 'qqp']:
       
    


                print(" k is {} learning rate is {} model name is {} task_name is {} ".format(k, lr, model, task_name))
                new_frame = result_frame[(result_frame['learning_rate']==lr)&(result_frame['model_name_or_path']==model)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
                display(new_frame)



                for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                    for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                        if l in metric:

                            print(metric, round(new_frame[metric].mean() * 100, 1),  \
                                  "("+str(round(new_frame[metric].std()*100,1) )+")")



                            print('')

 k is 30 learning rate is 0.001 model name is t5-small task_name is sst-2 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,sst-2_test_eval_loss,sst-2_test_eval_acc,sst-2-clue_test_eval_loss,sst-2-clue_test_eval_acc
240,none,sst-2,none,-1,0.001,t5-small,30,1,50.0,4,True,16,False,50,1.261793,0.779817,1.568452,0.761905
275,none,sst-2,none,-1,0.001,t5-small,30,4,50.0,4,True,16,False,50,1.156251,0.764908,1.266595,0.771429
484,none,sst-2,none,-1,0.001,t5-small,30,5,50.0,4,True,16,False,50,1.307361,0.771789,1.651018,0.785714
708,none,sst-2,none,-1,0.001,t5-small,30,3,50.0,4,True,16,False,50,1.180207,0.798165,1.242869,0.809524
758,none,sst-2,none,-1,0.001,t5-small,30,2,50.0,4,True,16,False,50,1.427811,0.755734,1.830866,0.785714


sst-2_test_eval_acc 77.4 (1.6)

sst-2-clue_test_eval_acc 78.3 (1.8)

 k is 30 learning rate is 0.001 model name is t5-small task_name is subj 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
259,none,subj,none,-1,0.001,t5-small,30,3,50.0,4,1.250399,0.8185,1.443474,0.814286,True,16,False,50
328,none,subj,none,-1,0.001,t5-small,30,2,50.0,4,0.842414,0.8595,0.959362,0.866667,True,16,False,50
379,none,subj,none,-1,0.001,t5-small,30,4,50.0,4,0.629916,0.8730,0.678209,0.861905,True,16,False,50
474,none,subj,none,-1,0.001,t5-small,30,5,50.0,4,1.019603,0.8360,0.855986,0.842857,True,16,False,50
554,none,subj,none,-1,0.001,t5-small,30,1,50.0,4,1.578462,0.8200,1.731108,0.833333,True,16,False,50


subj_test_eval_acc 84.1 (2.4)

subj-clue_test_eval_acc 84.4 (2.1)

 k is 30 learning rate is 0.001 model name is t5-small task_name is mpqa 


...

mnli_test_eval_mnli/acc 55.7 (1.9)

mnli-clue_test_eval_acc 52.7 (1.5)

mnli-mm_test_eval_mnli-mm/acc 57.3 (1.7)

 k is 30 learning rate is 5e-06 model name is t5-large task_name is rte 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
7,none,rte,none,-1,0.000005,t5-large,30,2,50.0,4,True,16,False,50,0.891850,0.588448,0.919402,0.576190
22,none,rte,none,-1,0.000005,t5-large,30,3,50.0,4,True,16,False,50,0.963302,0.610108,0.998466,0.614286
32,none,rte,none,-1,0.000005,t5-large,30,1,50.0,4,True,16,False,50,0.884841,0.606498,0.916126,0.585714
157,none,rte,none,-1,0.000005,t5-large,30,5,50.0,4,True,16,False,50,0.763544,0.667870,0.796121,0.661905
163,none,rte,none,-1,0.000005,t5-large,30,4,50.0,4,True,16,False,50,0.830046,0.638989,0.873792,0.619048


rte_test_eval_acc 62.2 (3.1)

rte-clue_test_eval_acc 61.1 (3.4)

 k is 30 learning rate is 5e-06 model name is t5-large task_name is qqp 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
2,none,qqp,none,-1,0.000005,t5-large,30,3,50.0,4,True,16,False,50,0.671919,0.745733,0.574644,0.660189,0.736207,0.709524
117,none,qqp,none,-1,0.000005,t5-large,30,4,50.0,4,True,16,False,50,0.689079,0.759411,0.618055,0.688733,0.718614,0.719048


qqp_test_eval_acc 75.3 (1.0)

qqp_test_eval_f1 59.6 (3.1)

qqp_test_eval_acc_and_f1 67.4 (2.0)

qqp_test_eval_acc_and_f1 67.4 (2.0)

qqp-clue_test_eval_acc 71.4 (0.7)



# adapter

In [141]:


result_list = []
task_list = ['CLUE_finetune_adapter_all_128']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



360


In [108]:
result_frame

,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,sst-2-clue_test_eval_loss,sst-2-clue_test_eval_acc,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc
0,adapter,mnli,linear_after,32,0.000005,128,roberta-large,10,1,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,adapter,mnli,linear_after,16,0.000005,128,roberta-large,10,1,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,adapter,mnli,linear_after,128,0.000005,128,roberta-large,20,1,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,adapter,mnli,linear_after,32,0.000005,128,roberta-large,20,1,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,adapter,mnli,linear_after,64,0.000005,128,roberta-large,20,1,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,adapter,subj,linear_after,32,0.000005,128,roberta-large,20,5,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.315465,0.9005,0.281624,0.895238
144,adapter,subj,linear_after,128,0.000005,128,roberta-large,20,5,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.475724,0.9020,0.507210,0.909524
145,adapter,subj,linear_after,32,0.000005,128,roberta-large,30,5,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.348987,0.9050,0.291166,0.890476
146,adapter,subj,linear_after,128,0.000005,128,roberta-large,30,5,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.533096,0.9035,0.520606,0.909524


In [146]:


for task_name in [ 'mnli', 'rte', 'qqp', 'sst-2', 'subj',  'mpqa']:
    for k in [ 30]:
  
        for adapter_dim in [128]:
        
            
       
    


            print(" k is {} task_name is {} adapter_dim is {}".format(k, task_name, adapter_dim))
            new_frame = result_frame[(result_frame['adapter_dim']==adapter_dim)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 30 task_name is mnli adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc
28,adapter,mnli,linear_after,128,0.000005,128,roberta-large,30,2,400.0,...,True,16,False,50,1.776239,0.640958,1.804640,0.647619,1.586107,0.667819
30,adapter,mnli,linear_after,128,0.000005,128,roberta-large,30,3,400.0,...,True,16,False,50,2.052726,0.627611,2.268697,0.642857,2.177687,0.627136
31,adapter,mnli,linear_after,128,0.000005,128,roberta-large,30,5,400.0,...,True,16,False,50,2.065706,0.627815,2.321686,0.585714,2.002635,0.636391
106,adapter,mnli,linear_after,128,0.000005,128,roberta-large,30,4,400.0,...,True,16,False,50,2.141597,0.629445,2.488990,0.614286,1.943255,0.651749
300,adapter,mnli,linear_after,128,0.000005,128,roberta-large,30,1,400.0,...,True,16,False,50,2.661117,0.595721,3.103591,0.571429,2.643332,0.602624


mnli_test_eval_mnli/acc 62.4 (1.7)

mnli-clue_test_eval_acc 61.2 (3.4)

mnli-mm_test_eval_mnli-mm/acc 63.7 (2.5)

 k is 30 task_name is rte adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
140,adapter,rte,linear_after,128,0.000005,128,roberta-large,30,4,400.0,4,True,16,False,50,1.816730,0.657040,1.722976,0.661905
159,adapter,rte,linear_after,128,0.000005,128,roberta-large,30,1,400.0,4,True,16,False,50,1.787105,0.638989,1.815468,0.647619
203,adapter,rte,linear_after,128,0.000005,128,roberta-large,30,5,400.0,4,True,16,False,50,1.692899,0.725632,1.567212,0.738095
299,adapter,rte,linear_after,128,0.000005,128,roberta-large,30,2,400.0,4,True,16,False,50,1.554982,0.664260,1.513204,0.676190
310,adapter,rte,linear_after,128,0.000005,128,roberta-large,30,3,400.0,4,True,16,False,50,1.799739,0.624549,1.795209,0.614286


rte_test_eval_acc 66.2 (3.9)

rte-clue_test_eval_acc 66.8 (4.6)

 k is 30 task_name is qqp adapter_dim is 128


...

sst-2_test_eval_acc 91.7 (1.0)

sst-2-clue_test_eval_acc 89.1 (1.2)

 k is 30 task_name is subj adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc
105,adapter,subj,linear_after,128,0.000005,128,roberta-large,30,3,400.0,4,True,16,False,50,0.658723,0.8925,0.621145,0.900000
184,adapter,subj,linear_after,128,0.000005,128,roberta-large,30,4,400.0,4,True,16,False,50,0.452998,0.9110,0.620475,0.876190
308,adapter,subj,linear_after,128,0.000005,128,roberta-large,30,5,400.0,4,True,16,False,50,0.533096,0.9035,0.520606,0.909524
312,adapter,subj,linear_after,128,0.000005,128,roberta-large,30,2,400.0,4,True,16,False,50,0.508765,0.9120,0.570675,0.904762
315,adapter,subj,linear_after,128,0.000005,128,roberta-large,30,1,400.0,4,True,16,False,50,0.409065,0.9270,0.444875,0.928571


subj_test_eval_acc 90.9 (1.3)

subj-clue_test_eval_acc 90.4 (1.9)

 k is 30 task_name is mpqa adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
40,adapter,mpqa,linear_after,128,0.000005,128,roberta-large,30,3,400.0,4,3.275034,0.6800,3.360449,0.661905,True,16,False,50
73,adapter,mpqa,linear_after,128,0.000005,128,roberta-large,30,1,400.0,4,1.613068,0.8075,1.545654,0.823810,True,16,False,50
117,adapter,mpqa,linear_after,128,0.000005,128,roberta-large,30,4,400.0,4,1.795273,0.8220,1.961103,0.804762,True,16,False,50
236,adapter,mpqa,linear_after,128,0.000005,128,roberta-large,30,2,400.0,4,1.463742,0.8190,1.501422,0.800000,True,16,False,50
327,adapter,mpqa,linear_after,128,0.000005,128,roberta-large,30,5,400.0,4,1.586859,0.8300,1.580113,0.828571,True,16,False,50


mpqa_test_eval_acc 79.2 (6.3)

mpqa-clue_test_eval_acc 78.4 (6.9)



In [84]:
result_frame[(result_frame['model_name_or_path']==model)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)

,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
5,adapter,qqp,lora,128,0.000005,256,roberta-large,30,2,400.0,...,True,16,False,50,1.832724,0.687435,0.621545,0.654490,1.949148,0.685714
40,adapter,qqp,lora,128,0.000005,256,roberta-large,30,4,400.0,...,True,16,False,50,1.598475,0.724957,0.585167,0.655062,1.996398,0.695238
47,adapter,qqp,lora,128,0.000005,256,roberta-large,30,3,400.0,...,True,16,False,50,2.241622,0.722978,0.520507,0.621742,2.599121,0.704762
60,adapter,qqp,linear_after,128,0.000005,256,roberta-large,30,2,400.0,...,True,16,False,50,2.299340,0.678011,0.612259,0.645135,2.264336,0.676190
61,adapter,qqp,linear_after,128,0.000005,256,roberta-large,30,3,400.0,...,True,16,False,50,3.428367,0.704106,0.460080,0.582093,3.971996,0.695238
62,adapter,qqp,lora,128,0.000005,256,roberta-large,30,5,400.0,...,True,16,False,50,2.749247,0.623943,0.581134,0.602538,2.629819,0.652381
67,adapter,qqp,linear_after,128,0.000005,256,roberta-large,30,1,400.0,...,True,16,False,50,2.570060,0.717413,0.568689,0.643051,3.210226,0.709524
77,adapter,qqp,linear_after,128,0.000005,256,roberta-large,30,5,400.0,...,True,16,False,50,3.826674,0.602325,0.583881,0.593103,3.810976,0.604762
87,adapter,qqp,linear_after,128,0.000005,256,roberta-large,30,4,400.0,...,True,16,False,50,2.430067,0.714618,0.525927,0.620272,2.933311,0.700000
105,adapter,qqp,lora,128,0.000005,256,roberta-large,30,1,400.0,...,True,16,False,50,1.877423,0.722013,0.590550,0.656282,2.196890,0.723810


In [200]:



result_list = []
task_list = ['CLUE_ADAMETAST_SOFTHARD']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length", "soft_label",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



60


In [201]:


for task_name in [ 'mnli', 'rte', 'qqp', 'sst-2', 'subj',  'mpqa']:
    for k in [ 30]:
        for soft_label in [1, 0]:
  







            print(" k is {} task_name is {} soft_label is {} ".format(k, task_name, soft_label))
            new_frame = result_frame[(result_frame['soft_label']==soft_label)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 30 task_name is mnli soft_label is 1 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc
6,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,30,5,...,True,16,False,100,2.007008,0.706266,2.246022,0.695238,1.895283,0.719386
7,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,30,3,...,True,16,False,100,2.001618,0.672644,2.368975,0.638095,2.094721,0.673820
31,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,30,2,...,True,16,False,100,2.062498,0.679165,2.193407,0.680952,1.733124,0.710537
32,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,30,4,...,True,16,False,100,1.847655,0.706470,2.387764,0.676190,1.755830,0.717453
33,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,30,1,...,True,16,False,100,2.649638,0.649720,3.367355,0.628571,2.694252,0.658869


mnli_test_eval_mnli/acc 68.3 (2.4)

mnli-clue_test_eval_acc 66.4 (2.9)

mnli-mm_test_eval_mnli-mm/acc 69.6 (2.8)

 k is 30 task_name is mnli soft_label is 0 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc
8,adapter,mnli,linear_after,128,0.000005,128,0,roberta-large,30,4,...,True,16,False,100,1.852253,0.700255,2.378626,0.671429,1.761446,0.713995
11,adapter,mnli,linear_after,128,0.000005,128,0,roberta-large,30,1,...,True,16,False,100,2.622360,0.648293,3.336553,0.614286,2.677452,0.656326
14,adapter,mnli,linear_after,128,0.000005,128,0,roberta-large,30,5,...,True,16,False,100,2.015831,0.700662,2.254270,0.671429,1.922914,0.712978
30,adapter,mnli,linear_after,128,0.000005,128,0,roberta-large,30,3,...,True,16,False,100,2.034412,0.662863,2.394370,0.642857,2.129767,0.665582
57,adapter,mnli,linear_after,128,0.000005,128,0,roberta-large,30,2,...,True,16,False,100,1.981727,0.679980,2.156817,0.652381,1.693903,0.705757


mnli_test_eval_mnli/acc 67.8 (2.3)

mnli-clue_test_eval_acc 65.0 (2.4)

mnli-mm_test_eval_mnli-mm/acc 69.1 (2.8)

 k is 30 task_name is rte soft_label is 1 


...

subj_test_eval_acc 91.7 (1.8)

subj-clue_test_eval_acc 91.2 (2.6)

 k is 30 task_name is mpqa soft_label is 1 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc
3,adapter,mpqa,linear_after,128,0.000005,128,1,roberta-large,30,5,10000.0,4,True,16,False,100,1.181896,0.8750,1.149155,0.861905
13,adapter,mpqa,linear_after,128,0.000005,128,1,roberta-large,30,1,10000.0,4,True,16,False,100,1.313166,0.8540,1.351046,0.861905
46,adapter,mpqa,linear_after,128,0.000005,128,1,roberta-large,30,3,10000.0,4,True,16,False,100,1.993523,0.8090,1.895097,0.828571
50,adapter,mpqa,linear_after,128,0.000005,128,1,roberta-large,30,2,10000.0,4,True,16,False,100,1.417611,0.8510,1.505659,0.838095
52,adapter,mpqa,linear_after,128,0.000005,128,1,roberta-large,30,4,10000.0,4,True,16,False,100,1.665939,0.8345,1.848129,0.804762


mpqa_test_eval_acc 84.5 (2.5)

mpqa-clue_test_eval_acc 83.9 (2.4)

 k is 30 task_name is mpqa soft_label is 0 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc
2,adapter,mpqa,linear_after,128,0.000005,128,0,roberta-large,30,1,10000.0,4,True,16,False,100,1.275237,0.8555,1.291241,0.857143
40,adapter,mpqa,linear_after,128,0.000005,128,0,roberta-large,30,3,10000.0,4,True,16,False,100,2.052797,0.8020,1.990360,0.809524
48,adapter,mpqa,linear_after,128,0.000005,128,0,roberta-large,30,5,10000.0,4,True,16,False,100,1.266208,0.8750,1.279594,0.861905
54,adapter,mpqa,linear_after,128,0.000005,128,0,roberta-large,30,4,10000.0,4,True,16,False,100,1.644684,0.8350,1.851078,0.800000
56,adapter,mpqa,linear_after,128,0.000005,128,0,roberta-large,30,2,10000.0,4,True,16,False,100,1.377876,0.8580,1.507527,0.847619


mpqa_test_eval_acc 84.5 (2.8)

mpqa-clue_test_eval_acc 83.5 (2.8)



In [148]:





result_list = []
task_list = ['CLUE_finetune_adapter_all_dim128_largerate']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



91


In [149]:


for task_name in [ 'mnli', 'rte', 'qqp', 'sst-2', 'subj',  'mpqa']:
    for k in [ 30]:
  
        for adapter_dim in [128]:
        
            
       
    


            print(" k is {} task_name is {} adapter_dim is {}".format(k, task_name, adapter_dim))
            new_frame = result_frame[(result_frame['adapter_dim']==adapter_dim)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 30 task_name is mnli adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
2,adapter,mnli,linear_after,128,0.00005,128,roberta-large,30,1,50.0,...,2.638430,0.600611,3.168988,0.576190,2.624814,0.610354,True,16,False,50
5,adapter,mnli,linear_after,128,0.00005,128,roberta-large,30,2,50.0,...,1.892993,0.645950,1.916083,0.642857,1.750825,0.670972,True,16,False,50
8,adapter,mnli,linear_after,128,0.00005,128,roberta-large,30,3,50.0,...,2.400919,0.613551,2.510037,0.633333,2.576320,0.620728,True,16,False,50
11,adapter,mnli,linear_after,128,0.00005,128,roberta-large,30,4,50.0,...,2.019338,0.642894,2.414405,0.638095,1.848842,0.660700,True,16,False,50
14,adapter,mnli,linear_after,128,0.00005,128,roberta-large,30,5,50.0,...,2.189648,0.618339,2.510003,0.590476,2.189986,0.627136,True,16,False,50


mnli_test_eval_mnli/acc 62.4 (2.0)

mnli-clue_test_eval_acc 61.6 (3.1)

mnli-mm_test_eval_mnli-mm/acc 63.8 (2.6)

 k is 30 task_name is rte adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
32,adapter,rte,linear_after,128,0.00005,128,roberta-large,30,1,50.0,4,True,16,False,50,1.870447,0.617329,1.955846,0.614286
35,adapter,rte,linear_after,128,0.00005,128,roberta-large,30,2,50.0,4,True,16,False,50,1.858075,0.657040,1.817937,0.661905
38,adapter,rte,linear_after,128,0.00005,128,roberta-large,30,3,50.0,4,True,16,False,50,1.740634,0.642599,1.734477,0.642857
41,adapter,rte,linear_after,128,0.00005,128,roberta-large,30,4,50.0,4,True,16,False,50,2.137012,0.653430,2.110306,0.638095
44,adapter,rte,linear_after,128,0.00005,128,roberta-large,30,5,50.0,4,True,16,False,50,1.706418,0.714801,1.579372,0.723810


rte_test_eval_acc 65.7 (3.6)

rte-clue_test_eval_acc 65.6 (4.1)

 k is 30 task_name is qqp adapter_dim is 128


...

sst-2_test_eval_acc 91.6 (0.9)

sst-2-clue_test_eval_acc 89.1 (1.3)

 k is 30 task_name is subj adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc
78,adapter,subj,linear_after,128,0.00005,128,roberta-large,30,1,50.0,4,True,16,False,50,0.369657,0.9370,0.468680,0.928571
81,adapter,subj,linear_after,128,0.00005,128,roberta-large,30,2,50.0,4,True,16,False,50,0.571054,0.9215,0.606385,0.928571
84,adapter,subj,linear_after,128,0.00005,128,roberta-large,30,3,50.0,4,True,16,False,50,0.643170,0.9110,0.672838,0.909524
87,adapter,subj,linear_after,128,0.00005,128,roberta-large,30,4,50.0,4,True,16,False,50,0.448721,0.9315,0.594342,0.919048
90,adapter,subj,linear_after,128,0.00005,128,roberta-large,30,5,50.0,4,True,16,False,50,0.679816,0.9055,0.720346,0.909524


subj_test_eval_acc 92.1 (1.3)

subj-clue_test_eval_acc 91.9 (1.0)

 k is 30 task_name is mpqa adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc
62,adapter,mpqa,linear_after,128,0.00005,128,roberta-large,30,1,50.0,4,True,16,False,50,1.844098,0.7805,1.669019,0.800000
65,adapter,mpqa,linear_after,128,0.00005,128,roberta-large,30,2,50.0,4,True,16,False,50,2.292834,0.7870,2.487563,0.757143
69,adapter,mpqa,linear_after,128,0.00005,128,roberta-large,30,3,50.0,4,True,16,False,50,1.351962,0.8505,1.508039,0.838095
72,adapter,mpqa,linear_after,128,0.00005,128,roberta-large,30,4,50.0,4,True,16,False,50,1.759437,0.8360,1.919130,0.819048
75,adapter,mpqa,linear_after,128,0.00005,128,roberta-large,30,5,50.0,4,True,16,False,50,1.460767,0.8605,1.397317,0.861905


mpqa_test_eval_acc 82.3 (3.7)

mpqa-clue_test_eval_acc 81.5 (4.0)



In [152]:





result_list = []
task_list = ['CLUE_finetune_adapter_all_dim128']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



90


In [175]:


for task_name in [ 'mnli', 'rte', 'qqp', 'sst-2', 'subj',  'mpqa']:
    for k in [10, 20, 30]:
  
        for adapter_dim in [128]:
        
            
       
    


            print(" k is {} task_name is {} adapter_dim is {}".format(k, task_name, adapter_dim))
            new_frame = result_frame[(result_frame['adapter_dim']==adapter_dim)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 10 task_name is mnli adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc


rte_test_eval_acc nan (nan)

rte-clue_test_eval_acc nan (nan)

eval_accumulation_steps nan (nan)

mpqa_test_eval_acc nan (nan)

mpqa-clue_test_eval_acc nan (nan)

qqp_test_eval_acc nan (nan)

qqp_test_eval_f1 nan (nan)

qqp_test_eval_acc_and_f1 nan (nan)

qqp_test_eval_acc_and_f1 nan (nan)

qqp-clue_test_eval_acc nan (nan)

 k is 20 task_name is mnli adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc


rte_test_eval_acc nan (nan)

rte-clue_test_eval_acc nan (nan)

eval_accumulation_steps nan (nan)

mpqa_test_eval_acc nan (nan)

mpqa-clue_test_eval_acc nan (nan)

qqp_test_eval_acc nan (nan)

qqp_test_eval_f1 nan (nan)

qqp_test_eval_acc_and_f1 nan (nan)

qqp_test_eval_acc_and_f1 nan (nan)

qqp-clue_test_eval_acc nan (nan)

 k is 30 task_name is mnli adapter_dim is 128


...

rte_test_eval_acc nan (nan)

rte-clue_test_eval_acc nan (nan)

eval_accumulation_steps nan (nan)

mpqa_test_eval_acc nan (nan)

mpqa-clue_test_eval_acc nan (nan)

qqp_test_eval_acc nan (nan)

qqp_test_eval_f1 nan (nan)

qqp_test_eval_acc_and_f1 nan (nan)

qqp_test_eval_acc_and_f1 nan (nan)

qqp-clue_test_eval_acc nan (nan)

 k is 20 task_name is mpqa adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc


rte_test_eval_acc nan (nan)

rte-clue_test_eval_acc nan (nan)

eval_accumulation_steps nan (nan)

mpqa_test_eval_acc nan (nan)

mpqa-clue_test_eval_acc nan (nan)

qqp_test_eval_acc nan (nan)

qqp_test_eval_f1 nan (nan)

qqp_test_eval_acc_and_f1 nan (nan)

qqp_test_eval_acc_and_f1 nan (nan)

qqp-clue_test_eval_acc nan (nan)

 k is 30 task_name is mpqa adapter_dim is 128


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc


rte_test_eval_acc nan (nan)

rte-clue_test_eval_acc nan (nan)

eval_accumulation_steps nan (nan)

mpqa_test_eval_acc nan (nan)

mpqa-clue_test_eval_acc nan (nan)

qqp_test_eval_acc nan (nan)

qqp_test_eval_f1 nan (nan)

qqp_test_eval_acc_and_f1 nan (nan)

qqp_test_eval_acc_and_f1 nan (nan)

qqp-clue_test_eval_acc nan (nan)



In [248]:

result_list = []
task_list = ['CLUE_prompt_metast_explorates']

args_list = [  'update_component', 'soft_label', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length", "semi_learning_rate",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size', 'semi_learning_rate', 'meta_st_warmup']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



60


In [249]:


for task_name in [ 'mnli', 'rte', 'qqp']:
    for k in [ 30]:
        for soft_label in [1]:
  
            for semi_learning_rate in [2e-5]:
                for meta_st_warmup in [0]:

                    print(" k is {} task_name is {} soft_label is {} meta_st_warmup is {}".format(k, task_name, soft_label, meta_st_warmup ))
                    new_frame = result_frame[(result_frame['soft_label'] ==soft_label)&(result_frame['meta_st_warmup']==meta_st_warmup)&(result_frame['semi_learning_rate']==semi_learning_rate)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
                    display(new_frame)



                    for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                        for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                            if l in metric:

                                print(metric, round(new_frame[metric].mean() * 100, 1),  \
                                      "("+str(round(new_frame[metric].std()*100,1) )+")")



                                print('')

 k is 30 task_name is mnli soft_label is 1 meta_st_warmup is 0


,update_component,soft_label,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,semi_learning_rate,model_name_or_path,num_k,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc
4,none,1,mnli,none,-1,0.000005,128,0.00002,roberta-large,30,...,True,16,False,100,6.400120,0.413449,6.372884,0.409524,5.815877,0.447010
12,none,1,mnli,none,-1,0.000005,128,0.00002,roberta-large,30,...,True,16,False,100,2.120252,0.749058,2.801610,0.728571,2.019723,0.760273
18,none,1,mnli,none,-1,0.000005,128,0.00002,roberta-large,30,...,True,16,False,100,2.383436,0.710036,2.816020,0.676190,2.475605,0.709418
26,none,1,mnli,none,-1,0.000005,128,0.00002,roberta-large,30,...,True,16,False,100,1.674142,0.763933,1.928075,0.738095,1.508583,0.783767
49,none,1,mnli,none,-1,0.000005,128,0.00002,roberta-large,30,...,True,16,False,100,2.577105,0.699643,3.465824,0.652381,2.343512,0.712368


mnli_test_eval_mnli/acc 66.7 (14.4)

mnli-clue_test_eval_acc 64.1 (13.4)

mnli-mm_test_eval_mnli-mm/acc 68.3 (13.5)

 k is 30 task_name is rte soft_label is 1 meta_st_warmup is 0


,update_component,soft_label,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,semi_learning_rate,model_name_or_path,num_k,...,per_device_train_batch_size,meta_st_warmup,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
1,none,1,rte,none,-1,0.000005,128,0.00002,roberta-large,30,...,4,0.0,2.509445,0.732852,2.582688,0.728571,True,16,False,100
9,none,1,rte,none,-1,0.000005,128,0.00002,roberta-large,30,...,4,0.0,2.764229,0.725632,2.770360,0.723810,True,16,False,100
14,none,1,rte,none,-1,0.000005,128,0.00002,roberta-large,30,...,4,0.0,3.010695,0.700361,3.169411,0.695238,True,16,False,100
24,none,1,rte,none,-1,0.000005,128,0.00002,roberta-large,30,...,4,0.0,2.810986,0.693141,2.758320,0.700000,True,16,False,100
38,none,1,rte,none,-1,0.000005,128,0.00002,roberta-large,30,...,4,0.0,2.563949,0.732852,2.452443,0.733333,True,16,False,100


rte_test_eval_acc 71.7 (1.9)

rte-clue_test_eval_acc 71.6 (1.7)

 k is 30 task_name is qqp soft_label is 1 meta_st_warmup is 0


,update_component,soft_label,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,semi_learning_rate,model_name_or_path,num_k,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
3,none,1,qqp,none,-1,0.000005,128,0.00002,roberta-large,30,...,True,16,False,100,2.190059,0.757977,0.671457,0.714717,2.000808,0.747619
37,none,1,qqp,none,-1,0.000005,128,0.00002,roberta-large,30,...,True,16,False,100,2.928963,0.723547,0.591618,0.657583,3.156423,0.728571
42,none,1,qqp,none,-1,0.000005,128,0.00002,roberta-large,30,...,True,16,False,100,4.274164,0.632773,0.563311,0.598042,3.989458,0.661905
50,none,1,qqp,none,-1,0.000005,128,0.00002,roberta-large,30,...,True,16,False,100,3.117780,0.698194,0.498025,0.598110,3.863267,0.652381
59,none,1,qqp,none,-1,0.000005,128,0.00002,roberta-large,30,...,True,16,False,100,3.354265,0.711328,0.513282,0.612305,3.932741,0.642857


qqp_test_eval_acc 70.5 (4.6)

qqp_test_eval_f1 56.8 (6.9)

qqp_test_eval_acc_and_f1 63.6 (5.0)

qqp_test_eval_acc_and_f1 63.6 (5.0)

qqp-clue_test_eval_acc 68.7 (4.8)



# MetaST-adapter

In [258]:


result_list = []
task_list = ['CLUE_SEMI_ADAPTER_largerate_WARMUP']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length", 'soft_label',
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



60


In [261]:

for task_name in ['sst-2', 'subj',  'mpqa',  'mnli', 'rte', 'qqp']:
    for k in [30]:
        for soft_label in [0]:
       
    


            print(" k is {} task_name is {} soft_label is {} ".format(k, task_name, soft_label))
            new_frame = result_frame[(result_frame['soft_label']==soft_label)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 30 task_name is sst-2 soft_label is 0 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,sst-2_test_eval_loss,sst-2_test_eval_acc,sst-2-clue_test_eval_loss,sst-2-clue_test_eval_acc
9,adapter,sst-2,linear_after,128,0.00005,128,0,roberta-large,30,5,10000.0,4,True,16,False,100,0.681099,0.923165,0.705784,0.900000
25,adapter,sst-2,linear_after,128,0.00005,128,0,roberta-large,30,1,10000.0,4,True,16,False,100,0.981512,0.894495,1.094033,0.876190
36,adapter,sst-2,linear_after,128,0.00005,128,0,roberta-large,30,4,10000.0,4,True,16,False,100,0.720166,0.927752,0.792115,0.909524
37,adapter,sst-2,linear_after,128,0.00005,128,0,roberta-large,30,3,10000.0,4,True,16,False,100,0.844021,0.913991,0.870780,0.904762
52,adapter,sst-2,linear_after,128,0.00005,128,0,roberta-large,30,2,10000.0,4,True,16,False,100,0.753335,0.920872,0.728012,0.904762


sst-2_test_eval_acc 91.6 (1.3)

sst-2-clue_test_eval_acc 89.9 (1.3)

 k is 30 task_name is subj soft_label is 0 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc
10,adapter,subj,linear_after,128,0.00005,128,0,roberta-large,30,5,10000.0,4,True,16,False,100,1.214223,0.8950,1.507164,0.876190
15,adapter,subj,linear_after,128,0.00005,128,0,roberta-large,30,2,10000.0,4,True,16,False,100,0.472486,0.9425,0.541882,0.947619
21,adapter,subj,linear_after,128,0.00005,128,0,roberta-large,30,3,10000.0,4,True,16,False,100,0.830382,0.9065,0.799378,0.914286
24,adapter,subj,linear_after,128,0.00005,128,0,roberta-large,30,4,10000.0,4,True,16,False,100,0.383902,0.9495,0.495290,0.933333
57,adapter,subj,linear_after,128,0.00005,128,0,roberta-large,30,1,10000.0,4,True,16,False,100,0.606767,0.9425,0.844644,0.923810


subj_test_eval_acc 92.7 (2.5)

subj-clue_test_eval_acc 91.9 (2.7)

 k is 30 task_name is mpqa soft_label is 0 


...

mnli_test_eval_mnli/acc 75.2 (3.1)

mnli-clue_test_eval_acc 70.8 (5.1)

mnli-mm_test_eval_mnli-mm/acc 76.3 (3.6)

 k is 30 task_name is rte soft_label is 0 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
8,adapter,rte,linear_after,128,0.00005,128,0,roberta-large,30,5,10000.0,4,True,16,False,100,3.460046,0.732852,3.395640,0.723810
23,adapter,rte,linear_after,128,0.00005,128,0,roberta-large,30,2,10000.0,4,True,16,False,100,3.752425,0.660650,3.812927,0.647619
29,adapter,rte,linear_after,128,0.00005,128,0,roberta-large,30,4,10000.0,4,True,16,False,100,4.732354,0.653430,4.878542,0.628571
30,adapter,rte,linear_after,128,0.00005,128,0,roberta-large,30,1,10000.0,4,True,16,False,100,4.038250,0.678700,4.493592,0.657143
50,adapter,rte,linear_after,128,0.00005,128,0,roberta-large,30,3,10000.0,4,True,16,False,100,3.758428,0.682310,3.795299,0.666667


rte_test_eval_acc 68.2 (3.1)

rte-clue_test_eval_acc 66.5 (3.6)

 k is 30 task_name is qqp soft_label is 0 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
6,adapter,qqp,linear_after,128,0.00005,128,0,roberta-large,30,3,...,True,16,False,100,2.412728,0.759362,0.629386,0.694374,2.854837,0.733333
18,adapter,qqp,linear_after,128,0.00005,128,0,roberta-large,30,4,...,True,16,False,100,3.832958,0.659906,0.487552,0.573729,4.344844,0.623810
38,adapter,qqp,linear_after,128,0.00005,128,0,roberta-large,30,2,...,True,16,False,100,3.138590,0.730349,0.616801,0.673575,2.990551,0.757143
43,adapter,qqp,linear_after,128,0.00005,128,0,roberta-large,30,5,...,True,16,False,100,2.438991,0.741034,0.671230,0.706132,2.543219,0.742857
48,adapter,qqp,linear_after,128,0.00005,128,0,roberta-large,30,1,...,True,16,False,100,2.796800,0.746451,0.605837,0.676144,2.655462,0.780952


qqp_test_eval_acc 72.7 (3.9)

qqp_test_eval_f1 60.2 (6.9)

qqp_test_eval_acc_and_f1 66.5 (5.3)

qqp_test_eval_acc_and_f1 66.5 (5.3)

qqp-clue_test_eval_acc 72.8 (6.1)



In [245]:

    

result_list = []
task_list = ['CLUE_SEMI_ADAPTER_HARD_explorates']

args_list = ['meta_st_warmup', 'semi_learning_rate',  'update_component', 'task_name', 'meta_st_warmup', 'semi_learning_rate', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length", 'soft_label',
            'model_name_or_path',  'num_k',  'seed']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



60


In [246]:

for task_name in [ 'mnli', 'rte', 'qqp']:
    for k in [30]:
        for soft_label in [1]:
            for meta_st_warmup in [0, 0.6]:
                for semi_learning_rate in [1e-4]:
                
    

                    print(" k is {} meta_st_warmup is {} semi_learning_rate is {} task_name is {} soft_label is {} ".format(k, meta_st_warmup, semi_learning_rate, task_name, soft_label))
                    new_frame = result_frame[(result_frame['semi_learning_rate']==semi_learning_rate)&(result_frame['meta_st_warmup'] == meta_st_warmup)&(result_frame['soft_label']==soft_label)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
                    display(new_frame)



                    for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                        for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                            if l in metric:

                                print(metric, round(new_frame[metric].mean() * 100, 1),  \
                                      "("+str(round(new_frame[metric].std()*100,1) )+")")



                                print('')

 k is 30 meta_st_warmup is 0 semi_learning_rate is 0.0001 task_name is mnli soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,...,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
9,0.0,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.052214,0.775650,1.437738,0.709524,0.945613,0.793938,True,16,False,100
18,0.0,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.199069,0.754661,1.725648,0.704762,1.115163,0.771257,True,16,False,100
38,0.0,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.290492,0.729699,1.632522,0.647619,1.302617,0.720606,True,16,False,100
39,0.0,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.182507,0.753337,1.596133,0.723810,1.136265,0.766172,True,16,False,100
41,0.0,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.601516,0.699134,2.081876,0.633333,1.490281,0.717555,True,16,False,100


mnli_test_eval_mnli/acc 74.2 (2.9)

mnli-clue_test_eval_acc 68.4 (4.0)

mnli-mm_test_eval_mnli-mm/acc 75.4 (3.3)

 k is 30 meta_st_warmup is 0.6 semi_learning_rate is 0.0001 task_name is mnli soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,...,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
0,0.6,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.086252,0.757922,1.430610,0.690476,1.004312,0.773291,True,16,False,100
5,0.6,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.465086,0.704534,1.943664,0.638095,1.490964,0.708808,True,16,False,100
22,0.6,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.004954,0.769231,1.350181,0.709524,0.862609,0.797803,True,16,False,100
36,0.6,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.540898,0.710749,2.061874,0.657143,1.517530,0.719182,True,16,False,100
37,0.6,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.315273,0.734182,1.641873,0.700000,1.222811,0.750000,True,16,False,100


mnli_test_eval_mnli/acc 73.5 (2.8)

mnli-clue_test_eval_acc 67.9 (3.0)

mnli-mm_test_eval_mnli-mm/acc 75.0 (3.7)

 k is 30 meta_st_warmup is 0 semi_learning_rate is 0.0001 task_name is rte soft_label is 1 


...

rte_test_eval_acc 71.0 (2.4)

rte-clue_test_eval_acc 71.0 (2.2)

 k is 30 meta_st_warmup is 0 semi_learning_rate is 0.0001 task_name is qqp soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
15,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.703685,0.753327,0.646460,0.699894,1.666612,0.761905
30,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,2.149061,0.621642,0.290129,0.455886,2.206332,0.619048
52,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.613481,0.757037,0.682750,0.719894,1.734368,0.738095
58,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.739744,0.743285,0.603022,0.673153,1.695792,0.742857
59,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.897570,0.744991,0.573967,0.659479,2.312555,0.700000


qqp_test_eval_acc 72.4 (5.8)

qqp_test_eval_f1 55.9 (15.6)

qqp_test_eval_acc_and_f1 64.2 (10.6)

qqp_test_eval_acc_and_f1 64.2 (10.6)

qqp-clue_test_eval_acc 71.2 (5.7)

 k is 30 meta_st_warmup is 0.6 semi_learning_rate is 0.0001 task_name is qqp soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
2,0.6,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.462247,0.755132,0.673095,0.714114,1.527981,0.761905
6,0.6,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.621139,0.765768,0.633882,0.699825,1.887490,0.704762
7,0.6,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.665627,0.745115,0.651870,0.698492,1.511809,0.742857
28,0.6,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,2.037402,0.744546,0.571630,0.658088,2.449506,0.709524
46,0.6,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.507492,0.749765,0.625199,0.687482,1.644335,0.723810


qqp_test_eval_acc 75.2 (0.9)

qqp_test_eval_f1 63.1 (3.8)

qqp_test_eval_acc_and_f1 69.2 (2.1)

qqp_test_eval_acc_and_f1 69.2 (2.1)

qqp-clue_test_eval_acc 72.9 (2.4)



In [254]:

    

result_list = []
task_list = ['CLUE_SEMI_ADAPTER_explorates_different_warmup']

args_list = ['meta_st_warmup', 'semi_learning_rate',  'update_component', 'task_name', 'meta_st_warmup', 'semi_learning_rate', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length", 'soft_label',
            'model_name_or_path',  'num_k',  'seed']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



75


In [255]:

for task_name in [ 'mnli', 'rte', 'qqp']:
    for k in [30]:
        for soft_label in [1]:
            for meta_st_warmup in [0.2, 0.4, 0.6, 0.8, 1]:
                for semi_learning_rate in [1e-4]:
                
    

                    print(" k is {} meta_st_warmup is {} semi_learning_rate is {} task_name is {} soft_label is {} ".format(k, meta_st_warmup, semi_learning_rate, task_name, soft_label))
                    new_frame = result_frame[(result_frame['semi_learning_rate']==semi_learning_rate)&(result_frame['meta_st_warmup'] == meta_st_warmup)&(result_frame['soft_label']==soft_label)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
                    display(new_frame)



                    for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                        for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                            if l in metric:

                                print(metric, round(new_frame[metric].mean() * 100, 1),  \
                                      "("+str(round(new_frame[metric].std()*100,1) )+")")



                                print('')

 k is 30 meta_st_warmup is 0.2 semi_learning_rate is 0.0001 task_name is mnli soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,...,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
10,0.2,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.058093,0.760672,1.316860,0.723810,0.901243,0.790582,True,16,False,100
26,0.2,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.393911,0.694651,1.706880,0.647619,1.423501,0.698535,True,16,False,100
59,0.2,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.248350,0.756597,1.878898,0.695238,1.165184,0.769426,True,16,False,100
63,0.2,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.629090,0.685380,1.941732,0.623810,1.573983,0.699756,True,16,False,100
69,0.2,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.254229,0.750790,1.651124,0.719048,1.153904,0.765256,True,16,False,100


mnli_test_eval_mnli/acc 73.0 (3.6)

mnli-clue_test_eval_acc 68.2 (4.4)

mnli-mm_test_eval_mnli-mm/acc 74.5 (4.3)

 k is 30 meta_st_warmup is 0.4 semi_learning_rate is 0.0001 task_name is mnli soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,...,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
7,0.4,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.047121,0.769333,1.363504,0.714286,0.892657,0.796684,True,16,False,100
32,0.4,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.749751,0.692410,2.349278,0.638095,1.667833,0.702705,True,16,False,100
39,0.4,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.266972,0.756291,1.787042,0.700000,1.200965,0.770240,True,16,False,100
70,0.4,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.415480,0.709730,1.757663,0.676190,1.447417,0.703926,True,16,False,100
71,0.4,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,1.369715,0.723484,1.823482,0.685714,1.331548,0.730269,True,16,False,100


mnli_test_eval_mnli/acc 73.0 (3.2)

mnli-clue_test_eval_acc 68.3 (2.9)

mnli-mm_test_eval_mnli-mm/acc 74.1 (4.2)

 k is 30 meta_st_warmup is 0.6 semi_learning_rate is 0.0001 task_name is mnli soft_label is 1 


...

qqp_test_eval_acc 75.2 (0.9)

qqp_test_eval_f1 63.1 (3.8)

qqp_test_eval_acc_and_f1 69.2 (2.1)

qqp_test_eval_acc_and_f1 69.2 (2.1)

qqp-clue_test_eval_acc 72.9 (2.4)

 k is 30 meta_st_warmup is 0.8 semi_learning_rate is 0.0001 task_name is qqp soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
5,0.8,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.641194,0.743458,0.620768,0.682113,1.475278,0.738095
24,0.8,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.519815,0.764803,0.691956,0.728380,1.583507,0.761905
43,0.8,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.471730,0.767870,0.651361,0.709616,1.780428,0.709524
50,0.8,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.933878,0.723695,0.568704,0.646200,2.239905,0.695238
64,0.8,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.713052,0.752511,0.583916,0.668213,2.071445,0.723810


qqp_test_eval_acc 75.0 (1.8)

qqp_test_eval_f1 62.3 (5.0)

qqp_test_eval_acc_and_f1 68.7 (3.3)

qqp_test_eval_acc_and_f1 68.7 (3.3)

qqp-clue_test_eval_acc 72.6 (2.6)

 k is 30 meta_st_warmup is 1 semi_learning_rate is 0.0001 task_name is qqp soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
4,1.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.929014,0.732995,0.625810,0.679403,1.693319,0.728571
9,1.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.412931,0.763789,0.686639,0.725214,1.506139,0.757143
16,1.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.837521,0.735172,0.597375,0.666274,2.000853,0.709524
21,1.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.882869,0.749740,0.619910,0.684825,2.243412,0.695238
35,1.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,2.074016,0.743384,0.560214,0.651799,2.427486,0.704762


qqp_test_eval_acc 74.5 (1.2)

qqp_test_eval_f1 61.8 (4.6)

qqp_test_eval_acc_and_f1 68.2 (2.8)

qqp_test_eval_acc_and_f1 68.2 (2.8)

qqp-clue_test_eval_acc 71.9 (2.5)



In [272]:

    

result_list = []
task_list = ['CLUE_SEMI_ADAPTER_warmup_steps']

args_list = ['meta_st_warmup', 'semi_learning_rate',  'update_component', 'task_name', 'meta_st_warmup', 'semi_learning_rate', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length", 'soft_label',
            'model_name_or_path',  'num_k',  'seed']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



30


In [273]:

for task_name in [ 'mnli', 'rte', 'qqp']:
    for k in [30]:
        for soft_label in [1]:
            for meta_st_warmup in [0]:
                for semi_learning_rate in [1e-4]:
                
    

                    print(" k is {} meta_st_warmup is {} semi_learning_rate is {} task_name is {} soft_label is {} ".format(k, meta_st_warmup, semi_learning_rate, task_name, soft_label))
                    new_frame = result_frame[(result_frame['semi_learning_rate']==semi_learning_rate)&(result_frame['meta_st_warmup'] == meta_st_warmup)&(result_frame['soft_label']==soft_label)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
                    display(new_frame)



                    for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                        for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                            if l in metric:

                                print(metric, round(new_frame[metric].mean() * 100, 1),  \
                                      "("+str(round(new_frame[metric].std()*100,1) )+")")



                                print('')

 k is 30 meta_st_warmup is 0 semi_learning_rate is 0.0001 task_name is mnli soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc
4,0.0,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.304386,0.734590,1.641657,0.695238,1.289674,0.737693
11,0.0,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,2.313380,0.454712,2.653134,0.385714,2.146704,0.464300
14,0.0,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,0.961736,0.775446,1.088303,0.752381,0.878853,0.796684
17,0.0,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.873456,0.697606,2.530501,0.642857,1.807486,0.700773
18,0.0,0.0001,adapter,mnli,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.346906,0.734284,1.825612,0.700000,1.262402,0.749186


mnli_test_eval_mnli/acc 67.9 (12.9)

mnli-clue_test_eval_acc 63.5 (14.5)

mnli-mm_test_eval_mnli-mm/acc 69.0 (13.1)

 k is 30 meta_st_warmup is 0 semi_learning_rate is 0.0001 task_name is rte soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,num_k,seed,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
7,0.0,0.0001,adapter,rte,linear_after,128,0.000005,128,1,roberta-large,30,5,True,16,False,100,0.648388,0.613718,0.654955,0.604762
9,0.0,0.0001,adapter,rte,linear_after,128,0.000005,128,1,roberta-large,30,4,True,16,False,100,1.852066,0.725632,1.959992,0.709524
13,0.0,0.0001,adapter,rte,linear_after,128,0.000005,128,1,roberta-large,30,2,True,16,False,100,1.665587,0.718412,1.635177,0.709524
16,0.0,0.0001,adapter,rte,linear_after,128,0.000005,128,1,roberta-large,30,1,True,16,False,100,1.787750,0.696751,1.879068,0.690476
22,0.0,0.0001,adapter,rte,linear_after,128,0.000005,128,1,roberta-large,30,3,True,16,False,100,1.913469,0.703971,1.942616,0.700000


rte_test_eval_acc 69.2 (4.5)

rte-clue_test_eval_acc 68.3 (4.4)

 k is 30 meta_st_warmup is 0 semi_learning_rate is 0.0001 task_name is qqp soft_label is 1 


,meta_st_warmup,semi_learning_rate,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,model_name_or_path,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
8,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.840890,0.750754,0.591809,0.671282,2.117408,0.738095
15,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.710922,0.749567,0.671703,0.710635,1.534919,0.766667
19,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.796145,0.739970,0.588742,0.664356,1.922073,0.757143
20,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.487379,0.693767,0.535997,0.614882,1.577455,0.685714
21,0.0,0.0001,adapter,qqp,linear_after,128,0.000005,128,1,roberta-large,...,True,16,False,100,1.448814,0.763789,0.696170,0.729979,1.563080,0.757143


qqp_test_eval_acc 74.0 (2.7)

qqp_test_eval_f1 61.7 (6.6)

qqp_test_eval_acc_and_f1 67.8 (4.5)

qqp_test_eval_acc_and_f1 67.8 (4.5)

qqp-clue_test_eval_acc 74.1 (3.3)



In [278]:


    

result_list = []
task_list = ['CLUE_SEMI_ADAPTER_warmup_steps_remainingtasks']

args_list = ['meta_st_warmup', 'semi_learning_rate',  'semi_warmup_ratio', 'update_component', 'task_name', 'meta_st_warmup', 'semi_learning_rate', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length", 'soft_label',
            'model_name_or_path',  'num_k',  'seed']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



30


In [279]:

for task_name in ['sst-2', 'subj',  'mpqa']:
    for k in [30]:
        for soft_label in [1]:
            for meta_st_warmup in [0.6]:
                for semi_learning_rate in [1e-4]:
                    for semi_warmup_ratio in [0]:
                
    

                        print(" k is {} meta_st_warmup is {} semi_warmup_ratio is {} task_name is {} soft_label is {} ".format(k, meta_st_warmup, semi_warmup_ratio, task_name, soft_label))
                        new_frame = result_frame[(result_frame['semi_warmup_ratio']==semi_warmup_ratio)&(result_frame['semi_learning_rate']==semi_learning_rate)&(result_frame['meta_st_warmup'] == meta_st_warmup)&(result_frame['soft_label']==soft_label)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
                        display(new_frame)



                        for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                            for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                                if l in metric:

                                    print(metric, round(new_frame[metric].mean() * 100, 1),  \
                                          "("+str(round(new_frame[metric].std()*100,1) )+")")



                                    print('')

 k is 30 meta_st_warmup is 0.6 semi_warmup_ratio is 0 task_name is sst-2 soft_label is 1 


,meta_st_warmup,semi_learning_rate,semi_warmup_ratio,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,...,num_k,seed,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,sst-2_test_eval_loss,sst-2_test_eval_acc,sst-2-clue_test_eval_loss,sst-2-clue_test_eval_acc
2,0.6,0.0001,0.0,adapter,sst-2,linear_after,128,0.000005,128,1,...,30,3,True,16,False,100,0.459400,0.930046,0.544546,0.909524
8,0.6,0.0001,0.0,adapter,sst-2,linear_after,128,0.000005,128,1,...,30,5,True,16,False,100,0.401288,0.925459,0.497037,0.914286
13,0.6,0.0001,0.0,adapter,sst-2,linear_after,128,0.000005,128,1,...,30,2,True,16,False,100,0.424570,0.936927,0.634941,0.904762
14,0.6,0.0001,0.0,adapter,sst-2,linear_after,128,0.000005,128,1,...,30,4,True,16,False,100,0.407224,0.933486,0.484039,0.923810
15,0.6,0.0001,0.0,adapter,sst-2,linear_after,128,0.000005,128,1,...,30,1,True,16,False,100,0.514948,0.913991,0.570512,0.914286


sst-2_test_eval_acc 92.8 (0.9)

sst-2-clue_test_eval_acc 91.3 (0.7)

 k is 30 meta_st_warmup is 0.6 semi_warmup_ratio is 0 task_name is subj soft_label is 1 


,meta_st_warmup,semi_learning_rate,semi_warmup_ratio,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,...,num_k,seed,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
11,0.6,0.0001,0.0,adapter,subj,linear_after,128,0.000005,128,1,...,30,4,0.317007,0.9410,0.459670,0.909524,True,16,False,100
20,0.6,0.0001,0.0,adapter,subj,linear_after,128,0.000005,128,1,...,30,2,0.344938,0.9450,0.416572,0.942857,True,16,False,100
21,0.6,0.0001,0.0,adapter,subj,linear_after,128,0.000005,128,1,...,30,5,0.593125,0.9075,0.612073,0.900000,True,16,False,100
22,0.6,0.0001,0.0,adapter,subj,linear_after,128,0.000005,128,1,...,30,1,0.205410,0.9625,0.289907,0.952381,True,16,False,100
28,0.6,0.0001,0.0,adapter,subj,linear_after,128,0.000005,128,1,...,30,3,0.457656,0.9180,0.492012,0.919048,True,16,False,100


subj_test_eval_acc 93.5 (2.2)

subj-clue_test_eval_acc 92.5 (2.2)

 k is 30 meta_st_warmup is 0.6 semi_warmup_ratio is 0 task_name is mpqa soft_label is 1 


,meta_st_warmup,semi_learning_rate,semi_warmup_ratio,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,...,num_k,seed,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc
3,0.6,0.0001,0.0,adapter,mpqa,linear_after,128,0.000005,128,1,...,30,3,True,16,False,100,0.785782,0.8820,0.772662,0.885714
7,0.6,0.0001,0.0,adapter,mpqa,linear_after,128,0.000005,128,1,...,30,4,True,16,False,100,1.022593,0.8445,0.986321,0.842857
18,0.6,0.0001,0.0,adapter,mpqa,linear_after,128,0.000005,128,1,...,30,1,True,16,False,100,0.988497,0.8250,1.159657,0.804762
23,0.6,0.0001,0.0,adapter,mpqa,linear_after,128,0.000005,128,1,...,30,2,True,16,False,100,1.111536,0.8470,1.214803,0.828571
25,0.6,0.0001,0.0,adapter,mpqa,linear_after,128,0.000005,128,1,...,30,5,True,16,False,100,0.936812,0.8635,0.987969,0.852381


mpqa_test_eval_acc 85.2 (2.1)

mpqa-clue_test_eval_acc 84.3 (3.0)



In [324]:



    

result_list = []
task_list = ['CLUE_SEMI_ADAPTER_meta_st_more_fn']

args_list = ['meta_st_warmup', 'semi_learning_rate',  'semi_warmup_ratio', 'update_component', 'task_name', 'meta_st_warmup', 'semi_learning_rate', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length", 'soft_label',
            'model_name_or_path',  'num_k',  'seed']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



30


In [325]:

for task_name in ['mnli', 'rte', 'qqp', 'sst-2', 'subj',  'mpqa']:
    for k in [30]:
        for soft_label in [1]:
            for meta_st_warmup in [0.6]:
                for semi_learning_rate in [1e-4]:
                    for semi_warmup_ratio in [0]:
                
    

                        print(" k is {} meta_st_warmup is {} semi_warmup_ratio is {} task_name is {} soft_label is {} ".format(k, meta_st_warmup, semi_warmup_ratio, task_name, soft_label))
                        new_frame = result_frame[(result_frame['semi_warmup_ratio']==semi_warmup_ratio)&(result_frame['semi_learning_rate']==semi_learning_rate)&(result_frame['meta_st_warmup'] == meta_st_warmup)&(result_frame['soft_label']==soft_label)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
                        display(new_frame)



                        for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                            for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                                if l in metric:

                                    print(metric, round(new_frame[metric].mean() * 100, 1),  \
                                          "("+str(round(new_frame[metric].std()*100,1) )+")")



                                    print('')

 k is 30 meta_st_warmup is 0.6 semi_warmup_ratio is 0 task_name is mnli soft_label is 1 


,meta_st_warmup,semi_learning_rate,semi_warmup_ratio,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc
9,0.6,0.0001,0.0,adapter,mnli,linear_after,128,0.000005,128,1,...,True,16,False,100,2.078588,0.707998,2.992021,0.628571,2.086488,0.707079
11,0.6,0.0001,0.0,adapter,mnli,linear_after,128,0.000005,128,1,...,True,16,False,100,1.603328,0.751808,2.301869,0.709524,1.463222,0.774003
12,0.6,0.0001,0.0,adapter,mnli,linear_after,128,0.000005,128,1,...,True,16,False,100,1.376813,0.762710,1.556589,0.714286,1.162992,0.788141
13,0.6,0.0001,0.0,adapter,mnli,linear_after,128,0.000005,128,1,...,True,16,False,100,1.999082,0.711768,2.630163,0.642857,2.090485,0.705350
14,0.6,0.0001,0.0,adapter,mnli,linear_after,128,0.000005,128,1,...,True,16,False,100,1.766854,0.726643,2.164123,0.709524,1.676812,0.741151


mnli_test_eval_mnli/acc 73.2 (2.4)

mnli-clue_test_eval_acc 68.1 (4.2)

mnli-mm_test_eval_mnli-mm/acc 74.3 (3.8)

 k is 30 meta_st_warmup is 0.6 semi_warmup_ratio is 0 task_name is rte soft_label is 1 


,meta_st_warmup,semi_learning_rate,semi_warmup_ratio,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,...,num_k,seed,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
2,0.6,0.0001,0.0,adapter,rte,linear_after,128,0.000005,128,1,...,30,4,True,16,False,100,2.756833,0.703971,2.858335,0.700000
4,0.6,0.0001,0.0,adapter,rte,linear_after,128,0.000005,128,1,...,30,2,True,16,False,100,2.413439,0.682310,2.424140,0.690476
19,0.6,0.0001,0.0,adapter,rte,linear_after,128,0.000005,128,1,...,30,1,True,16,False,100,2.435091,0.675090,2.570609,0.666667
20,0.6,0.0001,0.0,adapter,rte,linear_after,128,0.000005,128,1,...,30,3,True,16,False,100,2.823512,0.671480,2.817620,0.676190
21,0.6,0.0001,0.0,adapter,rte,linear_after,128,0.000005,128,1,...,30,5,True,16,False,100,2.134733,0.754513,2.206295,0.747619


rte_test_eval_acc 69.7 (3.4)

rte-clue_test_eval_acc 69.6 (3.1)

 k is 30 meta_st_warmup is 0.6 semi_warmup_ratio is 0 task_name is qqp soft_label is 1 


...

sst-2_test_eval_acc 92.4 (1.5)

sst-2-clue_test_eval_acc 91.1 (1.3)

 k is 30 meta_st_warmup is 0.6 semi_warmup_ratio is 0 task_name is subj soft_label is 1 


,meta_st_warmup,semi_learning_rate,semi_warmup_ratio,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,...,num_k,seed,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
0,0.6,0.0001,0.0,adapter,subj,linear_after,128,0.000005,128,1,...,30,4,0.426311,0.9380,0.629657,0.914286,True,16,False,100
1,0.6,0.0001,0.0,adapter,subj,linear_after,128,0.000005,128,1,...,30,5,0.644155,0.9140,0.692028,0.900000,True,16,False,100
3,0.6,0.0001,0.0,adapter,subj,linear_after,128,0.000005,128,1,...,30,2,0.444341,0.9395,0.522311,0.933333,True,16,False,100
10,0.6,0.0001,0.0,adapter,subj,linear_after,128,0.000005,128,1,...,30,3,0.522472,0.9265,0.625119,0.919048,True,16,False,100
29,0.6,0.0001,0.0,adapter,subj,linear_after,128,0.000005,128,1,...,30,1,0.389099,0.9535,0.492691,0.933333,True,16,False,100


subj_test_eval_acc 93.4 (1.5)

subj-clue_test_eval_acc 92.0 (1.4)

 k is 30 meta_st_warmup is 0.6 semi_warmup_ratio is 0 task_name is mpqa soft_label is 1 


,meta_st_warmup,semi_learning_rate,semi_warmup_ratio,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,soft_label,...,num_k,seed,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc
5,0.6,0.0001,0.0,adapter,mpqa,linear_after,128,0.000005,128,1,...,30,3,True,16,False,100,1.052352,0.8730,1.123515,0.861905
6,0.6,0.0001,0.0,adapter,mpqa,linear_after,128,0.000005,128,1,...,30,1,True,16,False,100,1.419641,0.8400,1.468408,0.828571
26,0.6,0.0001,0.0,adapter,mpqa,linear_after,128,0.000005,128,1,...,30,2,True,16,False,100,1.212223,0.8535,1.409759,0.833333
27,0.6,0.0001,0.0,adapter,mpqa,linear_after,128,0.000005,128,1,...,30,4,True,16,False,100,1.451155,0.8450,1.572647,0.833333
28,0.6,0.0001,0.0,adapter,mpqa,linear_after,128,0.000005,128,1,...,30,5,True,16,False,100,1.312970,0.8565,1.399728,0.847619


mpqa_test_eval_acc 85.4 (1.3)

mpqa-clue_test_eval_acc 84.1 (1.4)



# Component

In [171]:






result_list = []
task_list = ['CLUE_prompt_finetune_component_extra_new_tasks']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



180


In [174]:
for k in [10]:
  
    for update_component in ['embedding', 'attention', 'feedforward-output', 'feedforward-intermediate']:

        for task_name in [   'qqp']:
    
        
            
       
    


            print(" k is {} task_name is {} update_component is {}".format(k, task_name, update_component))
            new_frame = result_frame[(result_frame['update_component']==update_component)& (result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 10 task_name is qqp update_component is embedding


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
17,embedding,qqp,none,-1,0.000005,128,roberta-large,10,2,50.0,...,True,16,False,50,0.885121,0.512243,0.278237,0.395240,0.828501,0.523810
27,embedding,qqp,none,-1,0.000005,128,roberta-large,10,5,50.0,...,True,16,False,50,0.847192,0.529310,0.258321,0.393815,0.777735,0.571429
53,embedding,qqp,none,-1,0.000005,128,roberta-large,10,1,50.0,...,True,16,False,50,0.883318,0.600247,0.145049,0.372648,0.806411,0.623810
78,embedding,qqp,none,-1,0.000005,128,roberta-large,10,4,50.0,...,True,16,False,50,0.865136,0.471383,0.310091,0.390737,0.823990,0.528571
167,embedding,qqp,none,-1,0.000005,128,roberta-large,10,3,50.0,...,True,16,False,50,0.929283,0.430670,0.420289,0.425480,0.923934,0.433333


qqp_test_eval_acc 50.9 (6.4)

qqp_test_eval_f1 28.2 (9.9)

qqp_test_eval_acc_and_f1 39.6 (1.9)

qqp_test_eval_acc_and_f1 39.6 (1.9)

qqp-clue_test_eval_acc 53.6 (7.0)

 k is 10 task_name is qqp update_component is attention


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
18,attention,qqp,none,-1,0.000005,128,roberta-large,10,5,50.0,...,True,16,False,50,1.565369,0.596785,0.516319,0.556552,1.456090,0.600000
70,attention,qqp,none,-1,0.000005,128,roberta-large,10,2,50.0,...,True,16,False,50,1.896650,0.587361,0.383458,0.485410,1.543946,0.628571
107,attention,qqp,none,-1,0.000005,128,roberta-large,10,3,50.0,...,True,16,False,50,1.759217,0.604947,0.450227,0.527587,2.125786,0.576190
130,attention,qqp,none,-1,0.000005,128,roberta-large,10,4,50.0,...,True,16,False,50,1.916684,0.458348,0.276616,0.367482,1.947798,0.452381
143,attention,qqp,none,-1,0.000005,128,roberta-large,10,1,50.0,...,True,16,False,50,1.512435,0.643235,0.461671,0.552453,1.485855,0.623810


qqp_test_eval_acc 57.8 (7.0)

qqp_test_eval_f1 41.8 (9.2)

qqp_test_eval_acc_and_f1 49.8 (7.8)

qqp_test_eval_acc_and_f1 49.8 (7.8)

qqp-clue_test_eval_acc 57.6 (7.2)

 k is 10 task_name is qqp update_component is feedforward-output


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
4,feedforward-output,qqp,none,-1,0.000005,128,roberta-large,10,1,50.0,...,True,16,False,50,1.529958,0.629434,0.437613,0.533523,1.505672,0.623810
93,feedforward-output,qqp,none,-1,0.000005,128,roberta-large,10,3,50.0,...,True,16,False,50,1.619337,0.644126,0.456729,0.550427,1.954610,0.609524
100,feedforward-output,qqp,none,-1,0.000005,128,roberta-large,10,5,50.0,...,True,16,False,50,1.566514,0.599876,0.480957,0.540417,1.401123,0.609524
115,feedforward-output,qqp,none,-1,0.000005,128,roberta-large,10,4,50.0,...,True,16,False,50,1.542793,0.500000,0.376677,0.438338,1.576995,0.490476
116,feedforward-output,qqp,none,-1,0.000005,128,roberta-large,10,2,50.0,...,True,16,False,50,2.080671,0.579718,0.398343,0.489030,1.663015,0.638095


qqp_test_eval_acc 59.1 (5.7)

qqp_test_eval_f1 43.0 (4.2)

qqp_test_eval_acc_and_f1 51.0 (4.7)

qqp_test_eval_acc_and_f1 51.0 (4.7)

qqp-clue_test_eval_acc 59.4 (5.9)

 k is 10 task_name is qqp update_component is feedforward-intermediate


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,qqp_test_eval_loss,qqp_test_eval_acc,qqp_test_eval_f1,qqp_test_eval_acc_and_f1,qqp-clue_test_eval_loss,qqp-clue_test_eval_acc
62,feedforward-intermediate,qqp,none,-1,0.000005,128,roberta-large,10,3,50.0,...,True,16,False,50,1.556902,0.622112,0.438433,0.530272,1.806667,0.595238
72,feedforward-intermediate,qqp,none,-1,0.000005,128,roberta-large,10,2,50.0,...,True,16,False,50,1.708335,0.561464,0.407222,0.484343,1.402468,0.614286
125,feedforward-intermediate,qqp,none,-1,0.000005,128,roberta-large,10,1,50.0,...,True,16,False,50,1.238298,0.637126,0.420233,0.528680,1.225126,0.633333
175,feedforward-intermediate,qqp,none,-1,0.000005,128,roberta-large,10,4,50.0,...,True,16,False,50,1.623114,0.453995,0.329822,0.391908,1.601045,0.461905
178,feedforward-intermediate,qqp,none,-1,0.000005,128,roberta-large,10,5,50.0,...,True,16,False,50,1.364353,0.577640,0.479517,0.528579,1.207461,0.614286


qqp_test_eval_acc 57.0 (7.2)

qqp_test_eval_f1 41.5 (5.5)

qqp_test_eval_acc_and_f1 49.3 (6.0)

qqp_test_eval_acc_and_f1 49.3 (6.0)

qqp-clue_test_eval_acc 58.4 (6.9)



In [ ]:
mpqa seed -2
mpqa seed -3
rte - 4
qqp-5

In [178]:
355 * 0.287

101.88499999999999

# conventional FN

In [280]:








result_list = []
task_list = ['CLUE_prompt_conventional_FN']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



360


In [283]:

for task_name in ['mnli', 'rte', 'qqp', 'sst-2', 'subj',  'mpqa']:
    for k in [30]:
        for model_name_or_path in ['roberta-large']:
       
    


            print(" k is {} task_name is {} model is {} ".format(k, task_name, model_name_or_path))
            new_frame = result_frame[ (result_frame['model_name_or_path']==model_name_or_path)&(result_frame['task_name']==task_name)&(result_frame['num_k']==k)].dropna(axis=1)
            display(new_frame)



            for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

                for l in ['acc', 'f1', 'spearmanr', 'matthews_correlation']:
                    if l in metric:

                        print(metric, round(new_frame[metric].mean() * 100, 1),  \
                              "("+str(round(new_frame[metric].std()*100,1) )+")")



                        print('')

 k is 30 task_name is mnli model is roberta-large 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,mnli_test_eval_loss,mnli_test_eval_mnli/acc,mnli-clue_test_eval_loss,mnli-clue_test_eval_acc,mnli-mm_test_eval_loss,mnli-mm_test_eval_mnli-mm/acc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
47,none,mnli,none,-1,0.000005,128,roberta-large,30,1,50.0,...,3.400267,0.366072,3.769416,0.328571,3.435920,0.369609,True,16,False,50
50,none,mnli,none,-1,0.000005,128,roberta-large,30,2,50.0,...,2.101806,0.406011,1.917281,0.428571,1.773179,0.443857,True,16,False,50
53,none,mnli,none,-1,0.000005,128,roberta-large,30,3,50.0,...,2.108386,0.367702,2.013405,0.342857,1.989015,0.373881,True,16,False,50
56,none,mnli,none,-1,0.000005,128,roberta-large,30,4,50.0,...,2.777435,0.372389,2.829032,0.338095,2.639202,0.376526,True,16,False,50
59,none,mnli,none,-1,0.000005,128,roberta-large,30,5,50.0,...,3.021343,0.387570,3.177815,0.357143,3.108464,0.383950,True,16,False,50


mnli_test_eval_mnli/acc 38.0 (1.7)

mnli-clue_test_eval_acc 35.9 (4.0)

mnli-mm_test_eval_mnli-mm/acc 39.0 (3.1)

 k is 30 task_name is rte model is roberta-large 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,rte_test_eval_loss,rte_test_eval_acc,rte-clue_test_eval_loss,rte-clue_test_eval_acc
167,none,rte,none,-1,0.000005,128,roberta-large,30,1,50.0,4,True,16,False,50,1.973439,0.516245,1.757932,0.552381
170,none,rte,none,-1,0.000005,128,roberta-large,30,2,50.0,4,True,16,False,50,2.821972,0.512635,2.741922,0.514286
173,none,rte,none,-1,0.000005,128,roberta-large,30,3,50.0,4,True,16,False,50,2.361169,0.566787,2.356387,0.576190
176,none,rte,none,-1,0.000005,128,roberta-large,30,4,50.0,4,True,16,False,50,2.740387,0.462094,2.613338,0.476190
179,none,rte,none,-1,0.000005,128,roberta-large,30,5,50.0,4,True,16,False,50,2.733261,0.512635,2.732196,0.509524


rte_test_eval_acc 51.4 (3.7)

rte-clue_test_eval_acc 52.6 (3.9)

 k is 30 task_name is qqp model is roberta-large 


...

sst-2_test_eval_acc 65.0 (11.5)

sst-2-clue_test_eval_acc 66.9 (5.2)

 k is 30 task_name is subj model is roberta-large 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,subj_test_eval_loss,subj_test_eval_acc,subj-clue_test_eval_loss,subj-clue_test_eval_acc
347,none,subj,none,-1,0.000005,128,roberta-large,30,1,50.0,4,True,16,False,50,0.465904,0.9275,0.479434,0.933333
350,none,subj,none,-1,0.000005,128,roberta-large,30,2,50.0,4,True,16,False,50,0.608764,0.9050,0.817113,0.890476
353,none,subj,none,-1,0.000005,128,roberta-large,30,3,50.0,4,True,16,False,50,0.580466,0.9170,0.662500,0.909524
356,none,subj,none,-1,0.000005,128,roberta-large,30,4,50.0,4,True,16,False,50,0.644608,0.8780,0.776044,0.861905
359,none,subj,none,-1,0.000005,128,roberta-large,30,5,50.0,4,True,16,False,50,0.737214,0.8820,0.853398,0.857143


subj_test_eval_acc 90.2 (2.2)

subj-clue_test_eval_acc 89.0 (3.2)

 k is 30 task_name is mpqa model is roberta-large 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mpqa_test_eval_loss,mpqa_test_eval_acc,mpqa-clue_test_eval_loss,mpqa-clue_test_eval_acc
287,none,mpqa,none,-1,0.000005,128,roberta-large,30,1,50.0,4,True,16,False,50,3.025551,0.6185,3.245719,0.557143
290,none,mpqa,none,-1,0.000005,128,roberta-large,30,2,50.0,4,True,16,False,50,3.871813,0.5490,4.050173,0.542857
293,none,mpqa,none,-1,0.000005,128,roberta-large,30,3,50.0,4,True,16,False,50,4.439545,0.5100,4.590238,0.485714
296,none,mpqa,none,-1,0.000005,128,roberta-large,30,4,50.0,4,True,16,False,50,3.622474,0.5130,3.841091,0.500000
299,none,mpqa,none,-1,0.000005,128,roberta-large,30,5,50.0,4,True,16,False,50,1.390072,0.6165,1.520778,0.585714


mpqa_test_eval_acc 56.1 (5.3)

mpqa-clue_test_eval_acc 53.4 (4.1)



# SKIP

In [299]:


result_list = []
task_list = ['full_adapter_skip_3']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



24


In [290]:
result_frame

,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,rte_dev_eval_loss,rte_dev_eval_acc,do_eval,per_device_eval_batch_size,per_gpu_eval_batch_size,eval_accumulation_steps,fp16_full_eval,eval_steps,cola_dev_eval_loss,cola_dev_eval_mcc
0,adapter,rte,linear_after,128,0.00010,256,roberta-base,-1,1,10.0,...,0.757689,0.537906,True,16,None,None,False,1000,NaN,NaN
1,adapter,cola,linear_after,128,0.00010,256,roberta-base,-1,1,10.0,...,NaN,NaN,True,16,None,None,False,1000,0.702394,0.075549
2,adapter,rte,linear_after,128,0.00010,256,bert-base-uncased,-1,1,10.0,...,0.707954,0.574007,True,16,None,None,False,1000,NaN,NaN
3,adapter,rte,linear_after,128,0.00002,256,roberta-base,-1,1,10.0,...,0.702650,0.541516,True,16,None,None,False,1000,NaN,NaN
4,adapter,rte,linear_after,128,0.00002,256,bert-base-uncased,-1,1,10.0,...,0.733830,0.541516,True,16,None,None,False,1000,NaN,NaN
5,adapter,rte,linear_after,128,0.00002,256,roberta-large,-1,1,10.0,...,0.696691,0.566787,True,16,None,None,False,1000,NaN,NaN
6,adapter,rte,linear_after,128,0.00010,256,bert-large-uncased,-1,1,10.0,...,0.642261,0.657040,True,16,None,None,False,1000,NaN,NaN
7,adapter,cola,linear_after,128,0.00010,256,bert-large-uncased,-1,1,10.0,...,NaN,NaN,True,16,None,None,False,1000,0.526865,0.352519
8,adapter,rte,linear_after,128,0.00002,256,bert-large-uncased,-1,1,10.0,...,0.657093,0.642599,True,16,None,None,False,1000,NaN,NaN
9,adapter,cola,linear_after,128,0.00002,256,bert-base-uncased,-1,1,10.0,...,NaN,NaN,True,16,None,None,False,1000,0.559601,0.253767


In [300]:

for task_name in ['mnli', 'rte', 'cola']:

    for model_name_or_path in ['bert-large-uncased', 'bert-base-uncased', 'roberta-base', 'roberta-large']:
       
    


        print(" k is {} task_name is {} model is {} ".format(k, task_name, model_name_or_path))
        new_frame = result_frame[ (result_frame['model_name_or_path']==model_name_or_path)&(result_frame['task_name']==task_name)].dropna(axis=1)
        display(new_frame)



        for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

            for l in ['acc', 'f1', 'spearmanr', 'mcc']:
                if l in metric:

                    print(metric, round(new_frame[metric].mean() * 100, 1),  \
                          "("+str(round(new_frame[metric].std()*100,1) )+")")



                    print('')

 k is 30 task_name is mnli model is bert-large-uncased 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,rte_dev_eval_loss,rte_dev_eval_acc,do_eval,per_device_eval_batch_size,per_gpu_eval_batch_size,eval_accumulation_steps,fp16_full_eval,eval_steps,cola_dev_eval_loss,cola_dev_eval_mcc


rte_dev_eval_acc nan (nan)

eval_accumulation_steps nan (nan)

cola_dev_eval_mcc nan (nan)

 k is 30 task_name is mnli model is bert-base-uncased 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,rte_dev_eval_loss,rte_dev_eval_acc,do_eval,per_device_eval_batch_size,per_gpu_eval_batch_size,eval_accumulation_steps,fp16_full_eval,eval_steps,cola_dev_eval_loss,cola_dev_eval_mcc


rte_dev_eval_acc nan (nan)

eval_accumulation_steps nan (nan)

cola_dev_eval_mcc nan (nan)

 k is 30 task_name is mnli model is roberta-base 


...

cola_dev_eval_mcc 29.3 (5.5)

 k is 30 task_name is cola model is roberta-base 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,cola_dev_eval_loss,cola_dev_eval_mcc
1,adapter,cola,linear_after,128,0.00010,256,roberta-base,-1,1,10.0,32,True,16,False,1000,0.702394,0.075549
11,adapter,cola,linear_after,128,0.00002,256,roberta-base,-1,1,10.0,32,True,16,False,1000,0.622014,0.000000


cola_dev_eval_mcc 3.8 (5.3)

 k is 30 task_name is cola model is roberta-large 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,cola_dev_eval_loss,cola_dev_eval_mcc
12,adapter,cola,linear_after,128,0.00002,256,roberta-large,-1,1,10.0,32,True,16,False,1000,0.626740,0.079800
15,adapter,cola,linear_after,128,0.00010,256,roberta-large,-1,1,10.0,32,True,16,False,1000,0.574507,0.510203


cola_dev_eval_mcc 29.5 (30.4)



In [302]:


result_list = []
task_list = ['full_adapter_skip_2']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



24


In [303]:

for task_name in ['mnli', 'rte', 'cola']: #'sst-2', 'subj',  'mpqa']:

    for model_name_or_path in ['bert-large-uncased', 'bert-base-uncased', 'roberta-base', 'roberta-large']:
       
    


        print(" k is {} task_name is {} model is {} ".format(k, task_name, model_name_or_path))
        new_frame = result_frame[ (result_frame['model_name_or_path']==model_name_or_path)&(result_frame['task_name']==task_name)].dropna(axis=1)
        display(new_frame)



        for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

            for l in ['acc', 'f1', 'spearmanr', 'mcc']:
                if l in metric:

                    print(metric, round(new_frame[metric].mean() * 100, 1),  \
                          "("+str(round(new_frame[metric].std()*100,1) )+")")



                    print('')

 k is 30 task_name is mnli model is bert-large-uncased 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,cola_dev_eval_loss,cola_dev_eval_mcc,do_eval,per_device_eval_batch_size,per_gpu_eval_batch_size,eval_accumulation_steps,fp16_full_eval,eval_steps,rte_dev_eval_loss,rte_dev_eval_acc


cola_dev_eval_mcc nan (nan)

eval_accumulation_steps nan (nan)

rte_dev_eval_acc nan (nan)

 k is 30 task_name is mnli model is bert-base-uncased 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,cola_dev_eval_loss,cola_dev_eval_mcc,do_eval,per_device_eval_batch_size,per_gpu_eval_batch_size,eval_accumulation_steps,fp16_full_eval,eval_steps,rte_dev_eval_loss,rte_dev_eval_acc


cola_dev_eval_mcc nan (nan)

eval_accumulation_steps nan (nan)

rte_dev_eval_acc nan (nan)

 k is 30 task_name is mnli model is roberta-base 


...

cola_dev_eval_mcc 29.3 (5.5)

 k is 30 task_name is cola model is roberta-base 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,cola_dev_eval_loss,cola_dev_eval_mcc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
0,adapter,cola,linear_after,128,0.00010,256,roberta-base,-1,1,10.0,32,0.611771,0.139504,True,16,False,1000
13,adapter,cola,linear_after,128,0.00002,256,roberta-base,-1,1,10.0,32,0.619817,0.000000,True,16,False,1000


cola_dev_eval_mcc 7.0 (9.9)

 k is 30 task_name is cola model is roberta-large 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,cola_dev_eval_loss,cola_dev_eval_mcc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
2,adapter,cola,linear_after,128,0.00010,256,roberta-large,-1,1,10.0,32,0.530831,0.401851,True,16,False,1000
9,adapter,cola,linear_after,128,0.00002,256,roberta-large,-1,1,10.0,32,0.641130,0.050264,True,16,False,1000


cola_dev_eval_mcc 22.6 (24.9)



In [332]:



result_list = []
task_list = ['full_adapter_skip_3_from_1']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



24


In [333]:

for task_name in ['mnli', 'rte', 'cola']: #'sst-2', 'subj',  'mpqa']:

    for model_name_or_path in ['bert-base-uncased', 'bert-large-uncased',  'roberta-base', 'roberta-large']:
       
    


        print(" k is {} task_name is {} model is {} ".format(k, task_name, model_name_or_path))
        new_frame = result_frame[ (result_frame['model_name_or_path']==model_name_or_path)&(result_frame['task_name']==task_name)].dropna(axis=1)
        display(new_frame)



        for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

            for l in ['acc', 'f1', 'spearmanr', 'mcc']:
                if l in metric:

                    print(metric, round(new_frame[metric].max() * 100, 1), \
                          metric, round(new_frame[metric].mean() * 100, 1),  \
                          "("+str(round(new_frame[metric].std()*100,1) )+")")



                    print('')

 k is 30 task_name is mnli model is bert-base-uncased 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mnli_dev_eval_loss,mnli_dev_eval_mnli/acc
8,adapter,mnli,linear_after,128,0.00002,256,bert-base-uncased,-1,1,10.0,32,True,16,False,1000,0.843888,0.615588
17,adapter,mnli,linear_after,128,0.00010,256,bert-base-uncased,-1,1,10.0,32,True,16,False,1000,0.781219,0.653591


mnli_dev_eval_mnli/acc 65.4 mnli_dev_eval_mnli/acc 63.5 (2.7)

 k is 30 task_name is mnli model is bert-large-uncased 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,mnli_dev_eval_loss,mnli_dev_eval_mnli/acc
3,adapter,mnli,linear_after,128,0.00010,256,bert-large-uncased,-1,1,10.0,32,True,16,False,1000,0.713986,0.695364
21,adapter,mnli,linear_after,128,0.00002,256,bert-large-uncased,-1,1,10.0,32,True,16,False,1000,0.768507,0.661233


mnli_dev_eval_mnli/acc 69.5 mnli_dev_eval_mnli/acc 67.8 (2.4)

 k is 30 task_name is mnli model is roberta-base 


...

cola_dev_eval_mcc 35.3 cola_dev_eval_mcc 30.8 (6.3)

 k is 30 task_name is cola model is roberta-base 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,cola_dev_eval_loss,cola_dev_eval_mcc
6,adapter,cola,linear_after,128,0.00010,256,roberta-base,-1,1,10.0,32,True,16,False,1000,0.492507,0.590595
16,adapter,cola,linear_after,128,0.00002,256,roberta-base,-1,1,10.0,32,True,16,False,1000,0.518394,0.373119


cola_dev_eval_mcc 59.1 cola_dev_eval_mcc 48.2 (15.4)

 k is 30 task_name is cola model is roberta-large 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps,cola_dev_eval_loss,cola_dev_eval_mcc
7,adapter,cola,linear_after,128,0.00002,256,roberta-large,-1,1,10.0,32,True,16,False,1000,0.514289,0.464812
14,adapter,cola,linear_after,128,0.00010,256,roberta-large,-1,1,10.0,32,True,16,False,1000,0.698074,0.570780


cola_dev_eval_mcc 57.1 cola_dev_eval_mcc 51.8 (7.5)



In [309]:




result_list = []
task_list = ['full_adapter_skip_3_add_from_1']

args_list = [  'update_component', 'task_name', 'adapter_choice', 'adapter_dim', 'learning_rate', "max_seq_length",
            'model_name_or_path',  'num_k',  'seed', 'num_train_epochs', 'per_device_train_batch_size']
#'clue_deberta_prompt_mnl_1', 'clue_deberta_prompt_mnl_k_10', 'clue_bert_prompt_mnli_1', 'clue_bert_prompt_mnli_2'
            
for task in task_list: 
  
    result_list += get_result_list(task, 'eval')
result_frame = pd.DataFrame(result_list).drop_duplicates()



24


In [310]:

for task_name in ['mnli', 'rte', 'cola']: #'sst-2', 'subj',  'mpqa']:

    for model_name_or_path in ['bert-large-uncased', 'bert-base-uncased', 'roberta-base', 'roberta-large']:
       
    


        print(" k is {} task_name is {} model is {} ".format(k, task_name, model_name_or_path))
        new_frame = result_frame[ (result_frame['model_name_or_path']==model_name_or_path)&(result_frame['task_name']==task_name)].dropna(axis=1)
        display(new_frame)



        for  metric in new_frame.columns[new_frame.columns.to_series().str.contains('eval')].to_series():

            for l in ['acc', 'f1', 'spearmanr', 'mcc']:
                if l in metric:

                    print(metric, round(new_frame[metric].max() * 100, 1), \
                          metric, round(new_frame[metric].mean() * 100, 1),  \
                          "("+str(round(new_frame[metric].std()*100,1) )+")")



                    print('')

 k is 30 task_name is mnli model is bert-large-uncased 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,cola_dev_eval_loss,cola_dev_eval_mcc,do_eval,per_device_eval_batch_size,per_gpu_eval_batch_size,eval_accumulation_steps,fp16_full_eval,eval_steps,rte_dev_eval_loss,rte_dev_eval_acc


cola_dev_eval_mcc nan cola_dev_eval_mcc nan (nan)

eval_accumulation_steps nan eval_accumulation_steps nan (nan)

rte_dev_eval_acc nan rte_dev_eval_acc nan (nan)

 k is 30 task_name is mnli model is bert-base-uncased 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,...,cola_dev_eval_loss,cola_dev_eval_mcc,do_eval,per_device_eval_batch_size,per_gpu_eval_batch_size,eval_accumulation_steps,fp16_full_eval,eval_steps,rte_dev_eval_loss,rte_dev_eval_acc


cola_dev_eval_mcc nan cola_dev_eval_mcc nan (nan)

eval_accumulation_steps nan eval_accumulation_steps nan (nan)

rte_dev_eval_acc nan rte_dev_eval_acc nan (nan)

 k is 30 task_name is mnli model is roberta-base 


...

cola_dev_eval_mcc 33.1 cola_dev_eval_mcc 29.3 (5.5)

 k is 30 task_name is cola model is roberta-base 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,cola_dev_eval_loss,cola_dev_eval_mcc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
4,adapter,cola,linear_after,128,0.00010,256,roberta-base,-1,1,10.0,32,0.582238,0.437319,True,16,False,1000
5,adapter,cola,linear_after,128,0.00002,256,roberta-base,-1,1,10.0,32,0.617632,0.000000,True,16,False,1000


cola_dev_eval_mcc 43.7 cola_dev_eval_mcc 21.9 (30.9)

 k is 30 task_name is cola model is roberta-large 


,update_component,task_name,adapter_choice,adapter_dim,learning_rate,max_seq_length,model_name_or_path,num_k,seed,num_train_epochs,per_device_train_batch_size,cola_dev_eval_loss,cola_dev_eval_mcc,do_eval,per_device_eval_batch_size,fp16_full_eval,eval_steps
6,adapter,cola,linear_after,128,0.00010,256,roberta-large,-1,1,10.0,32,0.636610,0.100826,True,16,False,1000
7,adapter,cola,linear_after,128,0.00002,256,roberta-large,-1,1,10.0,32,0.707039,0.020587,True,16,False,1000


cola_dev_eval_mcc 10.1 cola_dev_eval_mcc 6.1 (5.7)

